# Nouakchott, Mauritania City Scan Data Cleaning
##### Dicembre 2025

Basic data cleaning pipeline for appropriate CSV preparation necessary for City Scan JavaScript plots for Nouakchott, Mauritania

In [1]:
# standard library imports
import os
import sys

# add project root to python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# third-party imports
import numpy as np
import pandas as pd
import geopandas as gpd

# change to project root directory
os.chdir('../')
print("directory changes")
print(f"current working directory is:", os.getcwd())

# local imports (after changing directory)
from src.clean import clean_pg, clean_pas, clean_rwi_area, clean_uba, clean_uba_area, clean_lc, clean_pug, clean_pv, clean_pv_area, clean_aq_area, clean_summer_area, clean_ndvi_area, clean_deforestation_area, clean_flood, clean_e, clean_s, clean_ls_area, clean_ee, clean_l_area, clean_fwi


directory changes
current working directory is: /Users/carolinecullinan/dev/wb/nouakchott-mauritania


# POPULATION AND DEMOGRAPHIC TRENDS

### pg.csv preparation
### Observable Notebook functions/charts:
#### 1.) "plot_pga" / "chart_pga" ; and
#### 2.) "plot_pgp" / "chart_pg"

In [2]:
# POPULATION & DEMOGRAPHIC TRENDS - pg.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_pga" / "chart_pga" (absolute population growth); and 
# 2.) "plot_pgp" / "chart_pgp" (population growth percentage)

# load "raw" (i.e. "dirty") tabular output data
raw_df_pg = pd.read_csv('data/raw/population-growth.csv') # updatefile path

# basic info about raw data
print("Raw population growth data info:")
print(f"Shape: {raw_df_pg.shape}")
print(f"Columns: {list(raw_df_pg.columns)}")
print(f"Date range: {raw_df_pg['Year'].min()} - {raw_df_pg['Year'].max()}")
print(f"Data preview:")
print(raw_df_pg.head())
print("\n" + "="*50 + "\n")

# clean data using clean_pg function in clean.py
try:
    cleaned_df_pg = clean_pg('data/raw/population-growth.csv') # updatefile path
    print("Population growth data cleaned successfully!")
    
    # cleaned data info
    print(f"\nCleaned data shape: {cleaned_df_pg.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_pg.columns)}")
    print(f"Sample of cleaned data:")
    print(cleaned_df_pg.head(10))
    
    # basic data validation
    print(f"\nData validation:")
    print(f"- Missing values: {cleaned_df_pg.isnull().sum().sum()}")
    print(f"- Year range: {cleaned_df_pg['yearName'].min()} - {cleaned_df_pg['yearName'].max()}")
    print(f"- Population range: {cleaned_df_pg['population'].min():,} - {cleaned_df_pg['population'].max():,}")
    print(f"- Growth rate range: {cleaned_df_pg['populationGrowthPercentage'].min():.3f}% - {cleaned_df_pg['populationGrowthPercentage'].max():.3f}%")
    
    # check for any potential data quality issues
    if cleaned_df_pg['populationGrowthPercentage'].isna().sum() > 0:
        print(f"Note: {cleaned_df_pg['populationGrowthPercentage'].isna().sum()} missing growth rate values (expected for first year)")
    
except Exception as e:
    print(f"Error cleaning population growth data: {e}")
    print("Check that 'data/raw/population-growth.csv' exists and has the correct format")

# save cleaned data as csv file - pg.csv, and export
# (this is handled automatically by clean_pg function, but confirming)
if 'cleaned_df_pg' in locals():
    print(f"Cleaned data saved to: data/processed/pg.csv")
else:
    print("No cleaned data available to save")

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/population-growth.csv'

### pas.csv preparation
### Observable Notebook functions/charts:
#### 1.) "plot_pas" / "chart_pas"

In [4]:
# POPULATION AGE SEX - pas.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_pas" / "chart_pas" (population age sex, i.e., population by sex and age bracket, (i.e., Population Distribution by Age & Sex, xxxx))

# load "raw" (i.e. "dirty") tabular output data
raw_df_pas = pd.read_csv('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_demographics.csv') # updatefile path

# basic info about raw data
print("Raw population age structure data info:")
print(f"Shape: {raw_df_pas.shape}")
print(f"Columns: {list(raw_df_pas.columns)}")
print(f"Age groups: {sorted(raw_df_pas['age_group'].unique())}")
print(f"Sex categories: {raw_df_pas['sex'].unique()}")
print(f"Total population: {raw_df_pas['population'].sum():,.0f}")
print(f"Data preview:")
print(raw_df_pas.head())
print("\n" + "="*50 + "\n")

# clean data using clean_pas function in clean.py
try:
    cleaned_df_pas = clean_pas('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_demographics.csv') # updatefile path
    print(" Population age structure data cleaned successfully!")
    
    # cleaned data info
    print(f"\nCleaned data shape: {cleaned_df_pas.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_pas.columns)}")
    print(f"Sample of cleaned data:")
    print(cleaned_df_pas.head(10))
    
    # basic data validation
    print(f"\nData validation:")
    print(f"- Missing values: {cleaned_df_pas.isnull().sum().sum()}")
    print(f"- Age brackets: {sorted(cleaned_df_pas['ageBracket'].unique())}")
    print(f"- Sex categories: {sorted(cleaned_df_pas['sex'].unique())}")
    print(f"- Population count range: {cleaned_df_pas['count'].min():,.0f} - {cleaned_df_pas['count'].max():,.0f}")
    print(f"- Percentage range: {cleaned_df_pas['percentage'].min():.3f}% - {cleaned_df_pas['percentage'].max():.3f}%")
    print(f"- Year: {cleaned_df_pas['yearName'].iloc[0]}")
    
    # data quality checks
    total_percentage = cleaned_df_pas['percentage'].sum()
    print(f"- Total percentage sum: {total_percentage:.3f}% (should be ~100%)")
    
    if abs(total_percentage - 100) > 0.1:
        print(f"Warning: Percentage sum deviates from 100% by {abs(total_percentage - 100):.3f}%")
    
    # check for balanced sex representation
    sex_counts = cleaned_df_pas.groupby('sex')['count'].sum()
    print(f"- Population by sex: Female: {sex_counts.get('female', 0):,.0f}, Male: {sex_counts.get('male', 0):,.0f}")
    
    # check age bracket coverage
    expected_brackets = len(cleaned_df_pas['ageBracket'].unique())
    actual_records = len(cleaned_df_pas)
    print(f"- Age brackets: {expected_brackets}, Total records: {actual_records}")
    
    if actual_records != expected_brackets * 2:  # should be 2 records per age bracket (male/female)
        print(f"Note: Expected {expected_brackets * 2} records (2 per age bracket), found {actual_records}")
    
except Exception as e:
    print(f"Error cleaning population age structure data: {e}")
    print("Check that the demographics CSV file exists and has the correct format")
    print("Expected columns: age_group, sex, population")

# save cleaned data as csv file - pas.csv, and export
# (this is handled automatically by clean_pas function, but confirming)
if 'cleaned_df_pas' in locals():
    print(f"Cleaned data saved to: data/processed/pas.csv")
    
    # preview data structure
    print(f"\nData structure summary:")
    print(f"- Columns: {list(cleaned_df_pas.columns)}")
    print(f"- Records per sex: {len(cleaned_df_pas[cleaned_df_pas['sex'] == 'female'])}, {len(cleaned_df_pas[cleaned_df_pas['sex'] == 'male'])}")
    print(f"- Data types: {dict(cleaned_df_pas.dtypes)}")
else:
    print("No cleaned data available to save")

Raw population age structure data info:
Shape: (36, 3)
Columns: ['age_group', 'sex', 'population']
Age groups: ['0-1', '1-4', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '5-9', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80+']
Sex categories: ['f' 'm']
Total population: 1,345,253
Data preview:
  age_group sex    population
0       1-4   f  62353.000043
1       1-4   m  64829.461383
2       0-1   f  18782.104672
3       0-1   m  19528.068560
4       5-9   f  73314.980869


Cleaned data saved to: data/processed/pas.csv
Total population: 1,345,253
Age brackets: 17
Sex categories: 2
Total records: 34
 Population age structure data cleaned successfully!

Cleaned data shape: (34, 5)
Cleaned data columns: ['ageBracket', 'sex', 'count', 'percentage', 'yearName']
Sample of cleaned data:
  ageBracket     sex     count  percentage  yearName
0        0-4  female  81135.10    6.031215      2021
1        0-4    male  84357.53    6.270755      2021
2      10-14 

### RELATIVE WEALTH INDEX (RWI)

### rwi_area.csv preparation (i.e., Percentage of area with different relative wealth index levels - "Least wealthy", "Less wealthy", "Average wealth", "More wealthy", "Most wealthy")
### Observable Notebook functions/charts:
#### 1.) "plot_rwi_area" / "chart_rwi_area"

In [ ]:
# RELATIVE WEALTH INDEX (RWI) - rwi_area.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_rwi_area" / "chart_rwi_area" (i.e., Percentage of area with different relative wealth index levels - "Least wealthy", "Less wealthy", "Average wealth", "More wealthy", "Most wealthy")

# load "raw" (i.e. "dirty") tabular output data
input_gpkg_path = 'data/raw/2025-11-mauritania-nouakchott_02-process-output_spatial_nouakchott_rwi.gpkg'  # updatefile path

print("="*60)
print("RELATIVE WEALTH INDEX DATA PROCESSING")
print("="*60)

print("Analyzing GeoPackage data structure...")

try:
    gdf = gpd.read_file(input_gpkg_path)
    
    print(f"Columns: {list(gdf.columns)}")
    print(f"Total features: {len(gdf):,}")
    print(f"Geometry type: {gdf.geometry.type.unique()}")
    print(f"CRS: {gdf.crs}")
    
    # check for 'rwi' column
    if 'rwi' in gdf.columns:
        rwi_data = gdf['rwi'].dropna()
        print(f"\nRWI statistics:")
        print(f"- Range: {rwi_data.min():.3f} to {rwi_data.max():.3f}")
        print(f"- Mean: {rwi_data.mean():.3f}")
        print(f"- Median: {rwi_data.median():.3f}")
        print(f"- Standard deviation: {rwi_data.std():.3f}")
        print(f"- Valid values: {len(rwi_data):,}")
        print(f"- Null values: {gdf['rwi'].isna().sum()}")
    else:
        print(f"\nWarning: 'rwi' column not found. Available numeric columns: {gdf.select_dtypes(include=[np.number]).columns.tolist()}")

except Exception as e:
    print(f"Could not examine GeoPackage structure: {e}")

print("\n" + "-"*40)

# process gpkg file using clean_rwi_area function in clean.py
try:
    cleaned_df_rwi = clean_rwi_area(input_gpkg_path, rwi_column='rwi')
    print("\nRWI data processed successfully!")
    
    # display cleaned data structure
    print(f"\nCleaned data shape: {cleaned_df_rwi.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_rwi.columns)}")
    print(f"\nProcessed RWI data:")
    print(cleaned_df_rwi)
    
    # basic data validation
    print(f"\nData Validation:")
    print(f"- Missing values: {cleaned_df_rwi.isnull().sum().sum()}")
    print(f"- Wealth categories: {len(cleaned_df_rwi)}")
    print(f"- Total grid cells: {cleaned_df_rwi['count'].sum():,}")
    print(f"- Percentage sum: {cleaned_df_rwi['percentage'].sum():.1f}% (should be ~100%)")
    
    # wealth distribution analysis
    print(f"\nWealth Distribution Analysis:")
    
    total_cells = cleaned_df_rwi['count'].sum()
    
    for idx, row in cleaned_df_rwi.iterrows():
        print(f"- {row['bin']}: {row['count']:,} cells ({row['percentage']:.1f}% of area)")
    
    # wealth inequality metrics
    print(f"\nWealth Distribution Summary:")
    
    # least wealthy areas (bottom 20%)
    least_wealthy = cleaned_df_rwi[cleaned_df_rwi['bin'].isin(['Least wealthy'])]['percentage'].sum()

    # less wealthy areas (next 20%)
    less_wealthy = cleaned_df_rwi[cleaned_df_rwi['bin'].isin(['Less wealthy'])]['percentage'].sum()
    
    # average wealth (middle 20%)
    average_wealth = cleaned_df_rwi[cleaned_df_rwi['bin'] == 'Average wealth']['percentage'].sum()

    # more wealthy areas (next middle/top 20%)
    more_wealthy = cleaned_df_rwi[cleaned_df_rwi['bin'].isin(['More wealthy'])]['percentage'].sum()
    
    # most wealthy areas (top 20%)
    most_wealthly = cleaned_df_rwi[cleaned_df_rwi['bin'].isin(['Most wealthy'])]['percentage'].sum()
    
 
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values
    missing_values = cleaned_df_rwi.isnull().sum().sum()
    if missing_values > 0:
        print(f"Missing values detected: {missing_values}")
        quality_issues += 1
    
    # check for negative values in "count" / "percentage"
    negative_counts = (cleaned_df_rwi['count'] < 0).sum()
    negative_percentages = (cleaned_df_rwi['percentage'] < 0).sum()
    
    if negative_counts > 0:
        print(f"Negative count values: {negative_counts}")
        quality_issues += 1
    if negative_percentages > 0:
        print(f"Negative percentage values: {negative_percentages}")
        quality_issues += 1
    
    # check "percentage" sum
    percentage_sum = cleaned_df_rwi['percentage'].sum()
    if abs(percentage_sum - 100) > 0.1:
        print(f"Percentage sum deviation: {percentage_sum:.1f}% (should be ~100%)")
        quality_issues += 1
    
    # check for duplicate bins
    duplicates = cleaned_df_rwi['bin'].duplicated().sum()
    if duplicates > 0:
        print(f"Duplicate wealth categories: {duplicates}")
        quality_issues += 1
    
    # check for expected number of categories (should be 5)
    if len(cleaned_df_rwi) != 5:
        print(f"Unexpected number of categories: {len(cleaned_df_rwi)} (expected 5)")
        print(f"Note: This can happen if RWI values have many duplicates (especially with discrete vs continuous RWI values (e.g., 0, 1 vs. 0.0123, 0.943)) - Despite this, the percentages should always be correct because we're summing the actual grid cell areas - no data is discarded, just the bin boundaries might merge if needed.")
        quality_issues += 1
    
    if quality_issues == 0:
        print("No data quality issues detected")
    
except Exception as e:
    print(f"Error processing RWI data: {e}")
    print("Troubleshooting steps:")
    print("   1. Ensure GeoPackage file exists at the specified path")
    print("   2. Check that the file contains 'rwi' column")
    print("   3. Verify the GeoPackage file is not corrupted")
    print("   4. Ensure geopandas library is installed: pip install geopandas")
    print("   5. Check file permissions")

print("\n" + "="*60)

# BUILT FORM

### URBAN EXTENT AND CHANGE

### uba.csv preparation
### Observable Notebook functions/charts:
#### 1.) "plot_ubaa" / "chart_ubaa" ; and
#### 2.) "plot_ubap" / "chart_ubap"

In [5]:
# URBAN EXTENT AND CHANGE - uba.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_ubaa" / "chart_ubaa" (absolute urban extent and change)
# 2.) "plot_ubap" / "chart_ubap" (urban extent and change growth percentage)

# load "raw" (i.e. "dirty") tabular output data
raw_df_uba = pd.read_csv('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_wsf_stats.csv') # updatefile path

# basic info about raw data
print("Raw urban built area data info:")
print(f"Shape: {raw_df_uba.shape}")
print(f"Columns: {list(raw_df_uba.columns)}")
print(f"Year range: {raw_df_uba['year'].min()} - {raw_df_uba['year'].max()}")
print(f"UBA range: {raw_df_uba['cumulative sq km'].min():.2f} - {raw_df_uba['cumulative sq km'].max():.2f} sq km")
print(f"Total data points: {len(raw_df_uba)}")
print(f"Data preview:")
print(raw_df_uba.head())
print("\n" + "="*50 + "\n")

# clean data using clean_uba function in clean.py
try:
    cleaned_df_uba = clean_uba('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_wsf_stats.csv') # updatefile path
    print("Urban built area data cleaned successfully!")
    
    # cleaned data info
    print(f"\nCleaned data shape: {cleaned_df_uba.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_uba.columns)}")
    print(f"Sample of cleaned data:")
    print(cleaned_df_uba.head(10))
    
    # basic data validation
    print(f"\nData validation:")
    print(f"- Missing values: {cleaned_df_uba.isnull().sum().sum()}")
    print(f"- Year range: {cleaned_df_uba['yearName'].min()} - {cleaned_df_uba['yearName'].max()}")
    print(f"- UBA range: {cleaned_df_uba['uba'].min():.2f} - {cleaned_df_uba['uba'].max():.2f} sq km")
    print(f"- Growth rate range: {cleaned_df_uba['ubaGrowthPercentage'].min():.3f}% - {cleaned_df_uba['ubaGrowthPercentage'].max():.3f}%")
    print(f"- Total urban expansion: {cleaned_df_uba['uba'].max() - cleaned_df_uba['uba'].min():.2f} sq km over {cleaned_df_uba['yearName'].max() - cleaned_df_uba['yearName'].min()} years")
    
    # data quality checks
    print(f"\nUrban growth analysis:")
    # calculate average annual growth rate
    avg_growth = cleaned_df_uba['ubaGrowthPercentage'].mean()
    print(f"- Average annual UBA growth rate: {avg_growth:.3f}%")
    
    # check for any potential data quality issues
    if cleaned_df_uba['ubaGrowthPercentage'].isna().sum() > 0:
        print(f"Note: {cleaned_df_uba['ubaGrowthPercentage'].isna().sum()} missing growth rate values (expected for first year)")
    
    # check for negative growth (Note: urban area should generally increase)
    negative_growth = cleaned_df_uba[cleaned_df_uba['ubaGrowthPercentage'] < 0]
    if len(negative_growth) > 0:
        print(f"Warning: {len(negative_growth)} years with negative UBA growth detected")
        print(f"Years with decline: {negative_growth['yearName'].tolist()}")
      
except Exception as e:
    print(f"Error cleaning urban built area data: {e}")
    print("Check that the UBA CSV file exists and has the correct format")
    print("Expected columns: year, cumulative sq km")

# save cleaned data as csv file - uba.csv, and export
# (this is handled automatically by clean_uba function, but confirming)
if 'cleaned_df_uba' in locals():
    print(f"\nCleaned data saved to: data/processed/uba.csv")
    
    # preview data structure
    print(f"\nData structure summary:")
    print(f"- Columns: {list(cleaned_df_uba.columns)}")
    print(f"- Time series length: {len(cleaned_df_uba)} years")
    print(f"- Data types: {dict(cleaned_df_uba.dtypes)}")
else:
    print("No cleaned data available to save")

Raw urban built area data info:
Shape: (31, 2)
Columns: ['year', 'cumulative sq km']
Year range: 1985 - 2015
UBA range: 28.60 - 121.05 sq km
Total data points: 31
Data preview:
   year  cumulative sq km
0  1985         28.597767
1  1986         28.597767
2  1987         32.754955
3  1988         36.485323
4  1989         39.294931


Cleaned data saved to: data/processed/uba.csv
Years covered: 1985 - 2015
Total data points: 31
UBA range: 28.60 - 121.05 sq km
Urban built area data cleaned successfully!

Cleaned data shape: (31, 4)
Cleaned data columns: ['year', 'yearName', 'uba', 'ubaGrowthPercentage']
Sample of cleaned data:
   year  yearName    uba  ubaGrowthPercentage
0     1      1985  28.60                  NaN
1     2      1986  28.60                0.000
2     3      1987  32.75               14.510
3     4      1988  36.49               11.420
4     5      1989  39.29                7.673
5     6      1990  45.39               15.526
6     7      1991  45.56                0.375


### uba_area.csv preparation (i.e., % area with different years of built-up area urban expansion - "Before 1985","1986-1995","1996-2005", and "2006-2015")
### Observable Notebook functions/charts:
#### 1.) "plot_uba_area" / "chart_uba_area" (i.e., Percentage of area with different years of urban built-up area expansion, "Before 1985", "1986-1995", "1996-2005", "2006-2015")


In [ ]:
# URBAN EXTENT AND CHANGE - uba_area.csv preparation from raw tif data for Observable Notebook plot functions/charts:
# 1.) "plot_uba_area" / "chart_uba_area" (i.e., Percentage of rea with different years of urban built-up area expansion, "Before 1985", "1986-1995", "1996-2005", and "2006-2015")

# load "raw" (i.e. "dirty") tif data
input_tif_path = 'data/raw/cartagena_main_wsf_evolution_utm.tif'  # updatefile path

print("="*60)
print("URBAN BUILT-UP AREA EXPANSION DATA PROCESSING")
print("="*60)

# process tif file using clean_uba_area function in clean.py
try:
    cleaned_df_uba = clean_uba_area(input_tif_path)
    print("Urban expansion data processed successfully!")
    
    # cleaned data structure
    print(f"\nCleaned data shape: {cleaned_df_uba.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_uba.columns)}")
    print(f"\nProcessed Urban Expansion data:")
    print(cleaned_df_uba)
    
    # basic data validation
    print(f"\nData Validation:")
    print(f"- Missing values: {cleaned_df_uba.isnull().sum().sum()}")
    print(f"- Urban expansion periods: {len(cleaned_df_uba)}")
    print(f"- Total pixels: {cleaned_df_uba['count'].sum():,.0f}")
    print(f"- Percentage sum: {cleaned_df_uba['percentage'].sum():.1f}% (should be ~100%)")
    
    # urban expansion temporal analysis
    print(f"\nUrban Expansion Timeline:")
    
    total_pixels = cleaned_df_uba['count'].sum()
    
    for idx, row in cleaned_df_uba.iterrows():
        print(f"- {row['bin']}: {row['count']:,.0f} pixels ({row['percentage']:.1f}%)")
    
    # ID most and least active expansion periods
    if len(cleaned_df_uba) > 0:
        # filter out zero-count periods for meaningful analysis
        active_periods = cleaned_df_uba[cleaned_df_uba['count'] > 0]
        
        if len(active_periods) > 0:
            max_expansion = active_periods.loc[active_periods['percentage'].idxmax()]
            min_expansion = active_periods.loc[active_periods['percentage'].idxmin()]
            
            print(f"\n- Most active expansion period: {max_expansion['bin']} ({max_expansion['percentage']:.1f}%)")
            print(f"- Least active expansion period: {min_expansion['bin']} ({min_expansion['percentage']:.1f}%)")
    
    # historical vs recent development
    before_2000 = cleaned_df_uba[cleaned_df_uba['bin'].isin(['Before 1985', '1986-1995'])]['percentage'].sum()
    after_2000 = cleaned_df_uba[cleaned_df_uba['bin'].isin(['1996-2005', '2006-2015'])]['percentage'].sum()
    
    print(f"\nTemporal Distribution:")
    print(f"- Historical development (before 1996): {before_2000:.1f}%")
    print(f"- Recent development (1996-2015): {after_2000:.1f}%")
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values
    missing_values = cleaned_df_uba.isnull().sum().sum()
    if missing_values > 0:
        print(f"Missing values detected: {missing_values}")
        quality_issues += 1
    
    # check for negative values (should not exist for counts)
    negative_counts = (cleaned_df_uba['count'] < 0).sum()
    negative_percentages = (cleaned_df_uba['percentage'] < 0).sum()
    
    if negative_counts > 0:
        print(f"Negative count values: {negative_counts}")
        quality_issues += 1
    if negative_percentages > 0:
        print(f"Negative percentage values: {negative_percentages}")
        quality_issues += 1
    
    # check percentage sum
    percentage_sum = cleaned_df_uba['percentage'].sum()
    if abs(percentage_sum - 100) > 0.1:
        print(f"Percentage sum deviation: {percentage_sum:.1f}% (should be ~100%)")
        quality_issues += 1
    
    # check for duplicate periods
    duplicates = cleaned_df_uba['bin'].duplicated().sum()
    if duplicates > 0:
        print(f"Duplicate time periods: {duplicates}")
        quality_issues += 1
    
    # check for expected number of periods (should be 4)
    if len(cleaned_df_uba) != 4:
        print(f"Unexpected number of periods: {len(cleaned_df_uba)} (expected 4)")
        quality_issues += 1
    
    if quality_issues == 0:
        print("No data quality issues detected")
    
    # data structure summary
    print(f"\nData structure summary:")
    print(f"- Columns: {list(cleaned_df_uba.columns)}")
    print(f"- Time periods: {len(cleaned_df_uba)} expansion periods")
    print(f"- Data types: {dict(cleaned_df_uba.dtypes)}")
    
    print(f"\nOutput file ready: data/processed/uba_area.csv")

    
except Exception as e:
    print(f"Error processing urban expansion data: {e}")
    print("Troubleshooting steps:")
    print("   1. Ensure TIF file exists at the specified path")
    print("   2. Check that the TIF file contains valid year data (1900-2030 range)")
    print("   3. Verify the TIF file is not corrupted")
    print("   4. Ensure rasterio library is installed: pip install rasterio")
    print("   5. Check file permissions and disk space")
    print("   6. Verify year values in TIF are realistic (e.g., 1985, 2005, etc.)")

print("\n" + "="*60)

### LAND COVER

### lc.csv preparation
### Observable Notebook functions/charts:
#### 1.) "plot_lc" / "chart_lc

In [6]:
# LAND COVER - lc.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_lc" / "chart_lc" (land cover types)

# load "raw" (i.e. "dirty") tabular output data
raw_df_lc = pd.read_csv('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_lc.csv') # updatefile path

# basic info about raw data
print("Raw land cover data info:")
print(f"Shape: {raw_df_lc.shape}")
print(f"Columns: {list(raw_df_lc.columns)}")
print(f"Total data points: {len(raw_df_lc)}")

# preview land cover types and pixel counts
if 'Land Cover Type' in raw_df_lc.columns and 'Pixel Count' in raw_df_lc.columns:
    print(f"Land cover types: {raw_df_lc['Land Cover Type'].nunique()}")
    print(f"Total pixels: {raw_df_lc['Pixel Count'].sum():,.0f}")
    print(f"Pixel count range: {raw_df_lc['Pixel Count'].min():,.0f} - {raw_df_lc['Pixel Count'].max():,.0f}")

print(f"Data preview:")
print(raw_df_lc.head())
print("\n" + "="*50 + "\n")

# clean data using clean_lc function in clean.py
try:
    cleaned_df_lc = clean_lc('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_lc.csv') # updatefile path
    print("Land cover data cleaned successfully!")
    
    # cleaned data info
    print(f"\nCleaned data shape: {cleaned_df_lc.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_lc.columns)}")
    print(f"Sample of cleaned data:")
    print(cleaned_df_lc.head(10))
    
    # basic data validation
    print(f"\nData validation:")
    print(f"- Missing values: {cleaned_df_lc.isnull().sum().sum()}")
    print(f"- Land cover types: {len(cleaned_df_lc)}")
    print(f"- Total pixels: {cleaned_df_lc['pixelTotal'].iloc[0]:,.0f}")
    print(f"- Percentage sum: {cleaned_df_lc['percentage'].sum():.1f}% (should be ~100%)")
    
    # land cover analysis
    print(f"\nLand Cover Data Summary:")
    
    print(f"- Land cover types present: {len(cleaned_df_lc)}")
    print(f"- Pixel count range: {cleaned_df_lc['pixelCount'].min():,.0f} - {cleaned_df_lc['pixelCount'].max():,.0f}")
    
    # ID extremes
    dominant_type = cleaned_df_lc.iloc[0]  # first row after sorting by percentage
    least_common_type = cleaned_df_lc.iloc[-1]  # last row after sorting
    
    print(f"- Most common land cover: {dominant_type['lcType']} ({dominant_type['percentage']:.1f}%)")
    print(f"- Least common land cover: {least_common_type['lcType']} ({least_common_type['percentage']:.1f}%)")
    
    # coverage distribution
    above_10_percent = (cleaned_df_lc['percentage'] >= 10).sum()
    above_5_percent = (cleaned_df_lc['percentage'] >= 5).sum()
    above_1_percent = (cleaned_df_lc['percentage'] >= 1).sum()
    
    print(f"- Types with ≥10% coverage: {above_10_percent}")
    print(f"- Types with ≥5% coverage: {above_5_percent}")
    print(f"- Types with ≥1% coverage: {above_1_percent}")
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values in key columns
    missing_type = cleaned_df_lc['lcType'].isna().sum()
    missing_count = cleaned_df_lc['pixelCount'].isna().sum()
    missing_percentage = cleaned_df_lc['percentage'].isna().sum()
    
    if missing_type > 0:
        print(f"Missing land cover type values: {missing_type}")
        quality_issues += 1
    if missing_count > 0:
        print(f"Missing pixel count values: {missing_count}")
        quality_issues += 1
    if missing_percentage > 0:
        print(f"Missing percentage values: {missing_percentage}")
        quality_issues += 1
    
    # check for impossible values
    negative_pixels = (cleaned_df_lc['pixelCount'] < 0).sum()
    negative_percentage = (cleaned_df_lc['percentage'] < 0).sum()
    
    if negative_pixels > 0:
        print(f"Negative pixel count values: {negative_pixels}")
        quality_issues += 1
    if negative_percentage > 0:
        print(f"Negative percentage values: {negative_percentage}")
        quality_issues += 1
    
    # check percentage sum
    percentage_sum = cleaned_df_lc['percentage'].sum()
    if abs(percentage_sum - 100) > 0.1:
        print(f"Percentage sum deviation: {percentage_sum:.1f}% (should be ~100%)")
        quality_issues += 1
    
    # check for duplicate land cover types
    duplicates = cleaned_df_lc['lcType'].duplicated().sum()
    if duplicates > 0:
        print(f"Duplicate land cover types: {duplicates}")
        quality_issues += 1
    
    if quality_issues == 0:
        print("No data quality issues detected")
        
except Exception as e:
    print(f"Error cleaning land cover data: {e}")
    print("Check that the land cover CSV file exists and has the correct format")
    print("Expected columns: Land Cover Type, Pixel Count")

# save confirmation and next steps
if 'cleaned_df_lc' in locals():
    print(f"\nCleaned data saved to: data/processed/lc.csv")
    
    # preview of data structure
    print(f"\nData structure summary for Observable:")
    print(f"- Columns: {list(cleaned_df_lc.columns)}")
    print(f"- Data points: {len(cleaned_df_lc)} land cover types")
    print(f"- Data types: {dict(cleaned_df_lc.dtypes)}")
    print(f"- Coverage range: {cleaned_df_lc['percentage'].min():.1f}% - {cleaned_df_lc['percentage'].max():.1f}%")
    
else:
    print("No cleaned land cover data available")
    print("Troubleshooting steps:")
    print("   1. Ensure land cover CSV file exists in data/raw/")
    print("   2. Check file has correct columns: Land Cover Type, Pixel Count")
    print("   3. Verify pixel count values are numeric and positive")
    print("   4. Check that land cover types are properly named")

Raw land cover data info:
Shape: (11, 2)
Columns: ['Land Cover Type', 'Pixel Count']
Total data points: 11
Land cover types: 11
Total pixels: 4,406,066
Pixel count range: 0 - 2,913,519
Data preview:
  Land Cover Type    Pixel Count
0      Tree cover    6772.000000
1       Shrubland   18797.478431
2       Grassland   11182.137255
3        Cropland     274.000000
4        Built-up  969430.725490


Cleaned data saved to: data/processed/lc.csv
Land cover types: 8
Total pixels analyzed: 4,406,066
Percentage coverage verification: 100.0% (should be ~100%)
Dominant land cover: Bare / sparse vegetation (66.1%)
Land cover data cleaned successfully!

Cleaned data shape: (8, 4)
Cleaned data columns: ['lcType', 'pixelCount', 'pixelTotal', 'percentage']
Sample of cleaned data:
                     lcType  pixelCount    pixelTotal  percentage
0  Bare / sparse vegetation     2913519  4.406066e+06       66.13
1                  Built-up      969431  4.406066e+06       22.00
2    Permanent water bodies

# URBAN DEVELOPMENT DYNAMICS MATRIX

### pug.csv preparation
### Observable Notebook functions/charts:
#### 1.) "plot_uddm" / "chart_uddm"

In [ ]:
# URBAN DEVELOPMENT DYNAMICS MATRIX: POPULATION URBAN GROWTH RATIO - pug.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_uddm" / "chart_ud" (population vs urban growth analysis)

# prereqs: ensure pg.csv and uba.csv have been generated from clean_pg and clean_uba functions in clean.py
print("Checking prerequisite files...")

# check if required input files exist
import os
pg_file = 'data/processed/pg.csv'
uba_file = 'data/processed/uba.csv'

if os.path.exists(pg_file):
    print(f"Population growth file found: {pg_file}")
else:
    print(f"Population growth file missing: {pg_file}")
    print("Run clean_pg function first to generate this file")

if os.path.exists(uba_file):
    print(f"Urban built area file found: {uba_file}")
else:
    print(f"Urban built area file missing: {uba_file}")
    print("Run clean_uba function first to generate this file")

print("\n" + "="*50 + "\n")

# clean and merge data using clean_pug function in clean.py
try:
    cleaned_df_pug = clean_pug()  # uses default paths: pg.csv and uba.csv
    print("Population urban growth data merged and cleaned successfully!")
    
    # cleaned data info
    print(f"\nCleaned data shape: {cleaned_df_pug.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_pug.columns)}")
    print(f"Sample of cleaned data:")
    print(cleaned_df_pug.head(10))
    
    # basic data validation
    print(f"\nData validation:")
    print(f"- Missing values: {cleaned_df_pug.isnull().sum().sum()}")
    print(f"- Year range: {cleaned_df_pug['yearName'].min()} - {cleaned_df_pug['yearName'].max()}")
    print(f"- Population range: {cleaned_df_pug['population'].min():,} - {cleaned_df_pug['population'].max():,}")
    print(f"- UBA range: {cleaned_df_pug['uba'].min():.2f} - {cleaned_df_pug['uba'].max():.2f} sq km")
    print(f"- Density range: {cleaned_df_pug['density'].min():.1f} - {cleaned_df_pug['density'].max():.1f} people/sq km")
    
    # population vs urban growth analysis
    print(f"\nPopulation vs Urban Growth Analysis:")
    print(f"- Population growth rate range: {cleaned_df_pug['populationGrowthPercentage'].min():.3f}% - {cleaned_df_pug['populationGrowthPercentage'].max():.3f}%")
    print(f"- UBA growth rate range: {cleaned_df_pug['ubaGrowthPercentage'].min():.3f}% - {cleaned_df_pug['ubaGrowthPercentage'].max():.3f}%")
    
    # calculate averages (excluding "NaN" values)
    avg_pop_growth = cleaned_df_pug['populationGrowthPercentage'].mean()
    avg_uba_growth = cleaned_df_pug['ubaGrowthPercentage'].mean()
    print(f"- Average annual population growth: {avg_pop_growth:.3f}%")
    print(f"- Average annual UBA growth: {avg_uba_growth:.3f}%")
    
    # growth ratio analysis
    valid_ratios = cleaned_df_pug['populationUrbanGrowthRatio'].dropna()
    if len(valid_ratios) > 0:
        print(f"- Population/Urban growth ratio range: {valid_ratios.min():.3f} - {valid_ratios.max():.3f}")
        print(f"- Average growth ratio: {valid_ratios.mean():.3f}")
        
        # interpret growth patterns
        if valid_ratios.mean() > 1:
            print("Population growing faster than urban area (potential densification)")
        elif valid_ratios.mean() < 1:
            print("Urban area growing faster than population (potential sprawl)")
        else:
            print("Balanced population and urban growth")
    
    # density analysis
    print(f"\nUrban Density Analysis:")
    density_change = cleaned_df_pug['density'].iloc[-1] - cleaned_df_pug['density'].iloc[0]
    density_change_pct = (density_change / cleaned_df_pug['density'].iloc[0]) * 100
    print(f"- Starting density ({cleaned_df_pug['yearName'].iloc[0]}): {cleaned_df_pug['density'].iloc[0]:,.1f} people/sq km")
    print(f"- Ending density ({cleaned_df_pug['yearName'].iloc[-1]}): {cleaned_df_pug['density'].iloc[-1]:,.1f} people/sq km")
    print(f"- Total density change: {density_change:+.1f} people/sq km ({density_change_pct:+.1f}%)")
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    # check for missing ratios
    missing_ratios = cleaned_df_pug['populationUrbanGrowthRatio'].isna().sum()
    if missing_ratios > 0:
        print(f"Note: {missing_ratios} missing growth ratio values (possibly due to zero UBA growth)")
        zero_uba_growth = cleaned_df_pug[cleaned_df_pug['ubaGrowthPercentage'] == 0]
        if len(zero_uba_growth) > 0:
            print(f"Years with zero UBA growth: {zero_uba_growth['yearName'].tolist()}")
    
    # check for negative population growth
    negative_pop_growth = cleaned_df_pug[cleaned_df_pug['populationGrowthPercentage'] < 0]
    if len(negative_pop_growth) > 0:
        print(f"Note: {len(negative_pop_growth)} years with population decline")
        print(f"Decline years: {negative_pop_growth['yearName'].tolist()}")
    
    # check for negative urban growth
    negative_uba_growth = cleaned_df_pug[cleaned_df_pug['ubaGrowthPercentage'] < 0]
    if len(negative_uba_growth) > 0:
        print(f"Warning: {len(negative_uba_growth)} years with urban area decline")
        print(f"UBA decline years: {negative_uba_growth['yearName'].tolist()}")
    
except Exception as e:
    print(f"Error creating population urban growth data: {e}")
    print("Check that both pg.csv and uba.csv files exist and have the correct format")
    print("Expected pg.csv columns: yearName, population, populationGrowthPercentage")
    print("Expected uba.csv columns: yearName, uba, ubaGrowthPercentage")

# save confirmation and next steps
if 'cleaned_df_pug' in locals():
    print(f"\nCleaned data saved to: data/processed/pug.csv")
    
    # preview of data structure
    print(f"\nData structure summary:")
    print(f"- Columns: {list(cleaned_df_pug.columns)}")
    print(f"- Time series length: {len(cleaned_df_pug)} years")
    print(f"- Data types: {dict(cleaned_df_pug.dtypes)}")
    print(f"- Overlapping years between datasets: {len(cleaned_df_pug)} out of potential maximum")
    
    # summary statistics
    print(f"\nSummary statistics:")
    total_pop_growth = ((cleaned_df_pug['population'].iloc[-1] / cleaned_df_pug['population'].iloc[0]) - 1) * 100
    total_uba_growth = ((cleaned_df_pug['uba'].iloc[-1] / cleaned_df_pug['uba'].iloc[0]) - 1) * 100
    print(f"- Total population growth over period: {total_pop_growth:.1f}%")
    print(f"- Total urban area growth over period: {total_uba_growth:.1f}%")
    print(f"- NET population density change: {density_change_pct:+.1f}%")
    
else:
    print("No cleaned data available to save")
    print("Troubleshooting steps:")
    print("   1. Ensure pg.csv exists (run clean_pg function)")
    print("   2. Ensure uba.csv exists (run clean_uba function)")
    print("   3. Check that both files have overlapping years")

# CLIMATE CONDITIONS

### pv.csv preparation (i.e., monthly max pv potential)
### Observable Notebook functions/charts:
#### 1.) "plot_pv" / "chart_pv" (i.e., Seasonal availability of solar energy, January - December)
#### 2.) "plot_pv_alt" / "chart_pv_alt" (i.e., Seasonal availability of solar energy, January - December with colored conditions for "Excellent (4.5+)", "Favorable (3.5-4.5)", and "Less than Favorable (<3.5)" conditions)
#### 3.) "plot_pv_d" / "chart_pv_d" (i.e., Photovoltaic Potenital Condition Level Distribution)

In [ ]:
# PHOTOVOLTAIC POTENTIAL - pv.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_pv" / "chart_pv" (i.e., Seasonal availability of solar energy, January - December)
# 2.) "plot_pv_alt" / "chart_pv_alt" (i.e., Seasonal availability of solar energy, January - December with colored conditions for "Excellent (4.5+)", "Favorable (3.5-4.5)", and "Less than Favorable (<3.5)" conditions)
# 3.) "plot_pv_d" / "chart_pv_d" (i.e., Photovoltaic Potenital Condition Level Distribution)

# load "raw" (i.e. "dirty") tabular output data
raw_df_pv = pd.read_csv('data/raw/monthly-pv.csv') # updatefile path

# basic info about raw data
print("Raw photovoltaic potential data info:")
print(f"Shape: {raw_df_pv.shape}")
print(f"Columns: {list(raw_df_pv.columns)}")
print(f"Month range: {raw_df_pv['month'].min()} - {raw_df_pv['month'].max()}")
print(f"PV max range: {raw_df_pv['max'].min():.2f} - {raw_df_pv['max'].max():.2f}")
print(f"PV mean range: {raw_df_pv['mean'].min():.2f} - {raw_df_pv['mean'].max():.2f}")
print(f"Total data points: {len(raw_df_pv)}")
print(f"Data preview:")
print(raw_df_pv.head())
print("\n" + "="*50 + "\n")

# clean data using clean_pv function in clean.py
try:
    cleaned_df_pv = clean_pv('data/raw/monthly-pv.csv') # updatefile path
    print("Photovoltaic potential data cleaned successfully!")
    
    # cleaned data info
    print(f"\nCleaned data shape: {cleaned_df_pv.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_pv.columns)}")
    print(f"Sample of cleaned data:")
    print(cleaned_df_pv.head(12))  # show all 12 months
    
    # basic data validation
    print(f"\nData validation:")
    print(f"- Missing values: {cleaned_df_pv.isnull().sum().sum()}")
    print(f"- Month coverage: {len(cleaned_df_pv)} months (should be 12)")
    print(f"- Month range: {cleaned_df_pv['month'].min()} - {cleaned_df_pv['month'].max()}")
    print(f"- PV potential range: {cleaned_df_pv['maxPv'].min():.2f} - {cleaned_df_pv['maxPv'].max():.2f}")
    
    # solar energy analysis
    print(f"\nSolar Energy Analysis:")
    
    # ID peak and low months
    peak_month = cleaned_df_pv.loc[cleaned_df_pv['maxPv'].idxmax()]
    low_month = cleaned_df_pv.loc[cleaned_df_pv['maxPv'].idxmin()]
    
    print(f"- Peak solar month: {peak_month['monthName']} ({peak_month['maxPv']:.2f})")
    print(f"- Lowest solar month: {low_month['monthName']} ({low_month['maxPv']:.2f})")
    
    # seasonal analysis
    spring_months = cleaned_df_pv[cleaned_df_pv['month'].isin([3, 4, 5])]  # Mar, Apr, May
    summer_months = cleaned_df_pv[cleaned_df_pv['month'].isin([6, 7, 8])]  # Jun, Jul, Aug
    fall_months = cleaned_df_pv[cleaned_df_pv['month'].isin([9, 10, 11])]  # Sep, Oct, Nov
    winter_months = cleaned_df_pv[cleaned_df_pv['month'].isin([12, 1, 2])]  # Dec, Jan, Feb
    
    spring_avg = spring_months['maxPv'].mean()
    summer_avg = summer_months['maxPv'].mean()
    fall_avg = fall_months['maxPv'].mean()
    winter_avg = winter_months['maxPv'].mean()
    
    print(f"- Spring average (Mar-May): {spring_avg:.2f}")
    print(f"- Summer average (Jun-Aug): {summer_avg:.2f}")
    print(f"- Fall average (Sep-Nov): {fall_avg:.2f}")
    print(f"- Winter average (Dec-Feb): {winter_avg:.2f}")
    
    # calculate seasonal variations
    annual_avg = cleaned_df_pv['maxPv'].mean()
    peak_variation = ((cleaned_df_pv['maxPv'].max() - annual_avg) / annual_avg) * 100 # (i.e., "the best month 6% better than the average"
    low_variation = ((annual_avg - cleaned_df_pv['maxPv'].min()) / annual_avg) * 100 # (i.e., "the worst month 10% worse than the average")
    
    print(f"- Annual average: {annual_avg:.2f}")
    print(f"- Peak month deviation: +{peak_variation:.1f}% above average")
    print(f"- Low month deviation: -{low_variation:.1f}% below average")
    
    # energy planning insights
    summer_winter_ratio = summer_avg / winter_avg
    print(f"- Summer/Winter ratio: {summer_winter_ratio:.2f}x")
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    # check for complete month coverage
    expected_months = set(range(1, 13))
    actual_months = set(cleaned_df_pv['month'].unique())
    missing_months = expected_months - actual_months
    
    if missing_months:
        print(f"Warning: Missing months: {sorted(missing_months)}")
    else:
        print("Complete 12-month coverage")
    
    # check for reasonable PV values
    if cleaned_df_pv['maxPv'].min() < 0:
        print(f"Warning: Negative PV values detected (minimum: {cleaned_df_pv['maxPv'].min():.2f})")
    
    # ID unusual patterns
    monthly_diff = cleaned_df_pv['maxPv'].diff().abs()

    
except Exception as e:
    print(f"Error cleaning photovoltaic potential data: {e}")
    print("Check that the monthly-pv.csv file exists and has the correct format")
    print("Expected columns: month, max, min, mean")

# save cleaned data as csv file - pv.csv, and export
# (this is handled automatically by clean_pv function, but confirming)
if 'cleaned_df_pv' in locals():
    print(f"\nCleaned data saved to: data/processed/pv.csv")
    
    # preview of data structure
    print(f"\nData structure summary:")
    print(f"- Columns: {list(cleaned_df_pv.columns)}")
    print(f"- Time series type: Monthly (12 data points)")
    print(f"- Data types: {dict(cleaned_df_pv.dtypes)}")
    print(f"- Seasonal range: {summer_avg/winter_avg:.2f}x variation from winter to summer")
    
    # insights
    print(f"\nSolar Energy Data Summary:")
    print(f"- Highest solar months: {', '.join(cleaned_df_pv.nlargest(3, 'maxPv')['monthName'].tolist())}")
    print(f"- Lowest solar months: {', '.join(cleaned_df_pv.nsmallest(3, 'maxPv')['monthName'].tolist())}")
    
else:
    print("No cleaned data available to save")
    print("Troubleshooting steps:")
    print("   1. Ensure monthly-pv.csv exists in data/raw/")
    print("   2. Check file has correct columns: month, max, min, mean")
    print("   3. Verify data covers all 12 months")

### pv_area.csv preparation (i.e., % area with different pv conditions - "Excellent (4+5)","Favorable (3.5-4.5)","Less than Favorable (<3.5)")
### Observable Notebook functions/charts:
#### 1.) "plot_pv_area" / "chart_pv_area" (i.e., Percentage of Area with different photovoltaic potential conditions, "Excellent (4.5+)", "Favorable (3.5-4.5)", and "Less than Favorable (<3.5)")

In [ ]:
# PHOTOVOLTAIC POTENTIAL - pv_area.csv preparation from raw tif data for Observable Notebook plot functions/charts:
# 1.) "plot_pv_area" / "chart_pv_area" (i.e., Percentage of Area with different photovoltaic potential conditions, "Excellent (4.5+)", "Favorable (3.5-4.5)", and "Less than Favorable (<3.5)")

# load "raw" (i.e. "dirty") tif data
input_tif_path = 'data/raw/2025-04-colombia-cartagena_02-process-output_spatial_cartagena_solar.tif' # updatefile path

print("="*60)
print("photovoltaic potential by area data processing")
print("="*60)

# process tif file using clean_pv_area function in clean.py
try:
    cleaned_df_pv = clean_pv_area(input_tif_path)
    print("photovoltaic potential by area data processed successfully!")
    
    # cleaned data structure
    print(f"\nCleaned data shape: {cleaned_df_pv.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_pv.columns)}")
    print(f"\nProcessed PV data:")
    print(cleaned_df_pv)
    
    # basic data validation
    print(f"\nData Validation:")
    print(f"- Missing values: {cleaned_df_pv.isnull().sum().sum()}")
    print(f"- PV potential categories: {len(cleaned_df_pv)}")
    print(f"- Total pixels: {cleaned_df_pv['count'].sum():,.0f}")
    print(f"- Percentage sum: {cleaned_df_pv['percentage'].sum():.1f}% (should be ~100%)")
    
    # pv potential distribution analysis
    print(f"\nPV Potential Distribution:")
    
    # basic statistics
    total_pixels = cleaned_df_pv['count'].sum()
    
    for idx, row in cleaned_df_pv.iterrows():
        print(f"- {row['condition']} ({row['bin']}): {row['count']:,.0f} pixels ({row['percentage']:.1f}%)")
    
    # ID most and least favorable areas
    if len(cleaned_df_pv) > 0:
        max_coverage = cleaned_df_pv.loc[cleaned_df_pv['percentage'].idxmax()]
        min_coverage = cleaned_df_pv.loc[cleaned_df_pv['percentage'].idxmin()]
        
        print(f"\n- Dominant condition: {max_coverage['condition']} ({max_coverage['percentage']:.1f}%)")
        print(f"- Least common condition: {min_coverage['condition']} ({min_coverage['percentage']:.1f}%)")
    
    # coverage thresholds
    excellent_coverage = cleaned_df_pv[cleaned_df_pv['condition'] == 'Excellent']['percentage'].sum()
    favorable_coverage = cleaned_df_pv[cleaned_df_pv['condition'] == 'Favorable']['percentage'].sum()
    unfavorable_coverage = cleaned_df_pv[cleaned_df_pv['condition'] == 'Less than Favorable']['percentage'].sum()
    
    print(f"\nPV Potential Summary:")
    print(f"- Excellent potential areas: {excellent_coverage:.1f}%")
    print(f"- Favorable potential areas: {favorable_coverage:.1f}%")
    print(f"- Less favorable areas: {unfavorable_coverage:.1f}%")
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values
    missing_values = cleaned_df_pv.isnull().sum().sum()
    if missing_values > 0:
        print(f"Missing values detected: {missing_values}")
        quality_issues += 1
    
    # check for negative values (note: should NOT exist for pv potential)
    negative_counts = (cleaned_df_pv['count'] < 0).sum()
    negative_percentages = (cleaned_df_pv['percentage'] < 0).sum()
    
    if negative_counts > 0:
        print(f"Negative count values: {negative_counts}")
        quality_issues += 1
    if negative_percentages > 0:
        print(f"Negative percentage values: {negative_percentages}")
        quality_issues += 1
    
    # check percentage sum
    percentage_sum = cleaned_df_pv['percentage'].sum()
    if abs(percentage_sum - 100) > 0.1:
        print(f"Percentage sum deviation: {percentage_sum:.1f}% (should be ~100%)")
        quality_issues += 1
    
    # check for duplicate conditions
    duplicates = cleaned_df_pv['condition'].duplicated().sum()
    if duplicates > 0:
        print(f"Duplicate conditions: {duplicates}")
        quality_issues += 1
    
    if quality_issues == 0:
        print("No data quality issues detected")
    
    # data structure summary
    print(f"\nData structure summary:")
    print(f"- Columns: {list(cleaned_df_pv.columns)}")
    print(f"- Categories: {len(cleaned_df_pv)} PV potential levels")
    print(f"- Data types: {dict(cleaned_df_pv.dtypes)}")
    
    print(f"\n Output file ready: data/processed/pv_area.csv")
  
except Exception as e:
    print(f"Error processing pv potential data: {e}")
    print("Troubleshooting steps:")
    print("   1. Ensure TIF file exists at the specified path")
    print("   2. Check that the TIF file contains valid photovoltaic potential data")
    print("   3. Verify the TIF file is not corrupted")
    print("   4. Ensure rasterio library is installed: pip install rasterio")
    print("   5. Check file permissions and disk space")

print("\n" + "="*60)

### AIR QUALITY

### aq_area.csv preparation (i.e., % area with different air quality, 2019 concentration levels of PM2.5 (µg/mˆ3) - [0-5), [5-10), [10-15), [15-20), [20-30), [30-40), [40-50, [50-100), and [100+])
### Observable Notebook functions/charts:
#### 1.) "plot_aq_area" / "chart_aq_area" (i.e., Percentage of area with different air quality, 2019 concentration levels of PM2.5 (µg/mˆ3) - [0-5), [5-10), [10-15), [15-20), [20-30), [30-40), [40-50), [50-100), and [100+])

In [7]:
# AIR QUALITY - aq_area.csv preparation from raw tif data for Observable Notebook plot functions/charts:
# 1.) "plot_aq_area" / "chart_aq_area" (i.e., Percentage of Area with different air quality, 2019 concentration levels of PM2.5 (µg/mˆ3) - [0-5), [5-10), [10-15), [15-20), [20-30), [30-40), [40-50), [50-100), and [100+])

# load "raw" (i.e. "dirty") tif data
input_tif_path = 'data/raw/2025-11-mauritania-nouakchott_02-process-output_spatial_nouakchott_air.tif' # updatefile path

print("="*60)
print("AIR QUALITY (PM2.5) DATA PROCESSING")
print("="*60)

print("Analyzing TIF data structure...")

try:
    import rasterio
    with rasterio.open(input_tif_path) as src:
        data = src.read(1)
        # remove "NaN" and "NoData" values for analysis
        if src.nodata is not None:
            valid_data = data[data != src.nodata]
        else:
            valid_data = data[~np.isnan(data)]
        
        # remove negative values (i.e., shouldn't exist for PM2.5)
        valid_data = valid_data[valid_data >= 0]
        
        print(f"PM2.5 concentration range: {valid_data.min():.2f} - {valid_data.max():.2f}")
        print(f"NoData value: {src.nodata}")
        print(f"Total valid pixels: {len(valid_data):,}")
        print(f"Unique concentration values: {len(np.unique(valid_data)):,}")
        
        # basic statistics
        print(f"Mean concentration: {valid_data.mean():.2f} μg/m³")
        print(f"Median concentration: {np.median(valid_data):.2f} μg/m³")
        print(f"Standard deviation: {valid_data.std():.2f} μg/m³")

except Exception as e:
    print(f"Could not examine TIF structure: {e}")

print("\n" + "-"*40)

# process the tif file using clean_aq_area function in clean.py
try:
    cleaned_df_aq = clean_aq_area(input_tif_path)
    print("Air quality data processed successfully!")
    
    # cleaned data structure
    print(f"\nCleaned data shape: {cleaned_df_aq.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_aq.columns)}")
    print(f"\nProcessed Air Quality data:")
    print(cleaned_df_aq)
    
    # basic data validation
    print(f"\nData Validation:")
    print(f"- Missing values: {cleaned_df_aq.isnull().sum().sum()}")
    print(f"- Concentration bins: {len(cleaned_df_aq)}")
    print(f"- Total pixels: {cleaned_df_aq['count'].sum():,.0f}")
    print(f"- Percentage sum: {cleaned_df_aq['percentage'].sum():.1f}% (should be ~100%)")
    
    # PM2.5 concentration distribution analysis
    print(f"\nPM2.5 Concentration Distribution:")
    
    total_pixels = cleaned_df_aq['count'].sum()
    
    for idx, row in cleaned_df_aq.iterrows():
        if row['count'] > 0:  # only show bins with data
            print(f"- {row['bin']} μg/m³: {row['count']:,.0f} pixels ({row['percentage']:.1f}%)")
    
    # air quality level analysis
    if len(cleaned_df_aq) > 0:
        # filter out zero-count categories
        active_bins = cleaned_df_aq[cleaned_df_aq['count'] > 0]
        
        if len(active_bins) > 0:
            max_concentration_bin = active_bins.loc[active_bins['percentage'].idxmax()]
            min_concentration_bin = active_bins.loc[active_bins['percentage'].idxmin()]
            
            print(f"\n- Most common concentration range: {max_concentration_bin['bin']} μg/m³ ({max_concentration_bin['percentage']:.1f}%)")
            print(f"- Least common concentration range: {min_concentration_bin['bin']} μg/m³ ({min_concentration_bin['percentage']:.1f}%)")
    
    # air quality level groupings
    good_air = cleaned_df_aq[cleaned_df_aq['bin'].isin(['[0-5)', '[5-10)'])]['percentage'].sum()
    moderate_air = cleaned_df_aq[cleaned_df_aq['bin'].isin(['[10-15)', '[15-20)'])]['percentage'].sum()
    unhealthy_air = cleaned_df_aq[cleaned_df_aq['bin'].isin(['[20-30)', '[30-40)', '[40-50)', '[50-100)', '100+'])]['percentage'].sum()
    
    print(f"\nAir Quality Summary:")
    print(f"- Lower concentrations (0-10 μg/m³): {good_air:.1f}%")
    print(f"- Moderate concentrations (10-20 μg/m³): {moderate_air:.1f}%")
    print(f"- Higher concentrations (20+ μg/m³): {unhealthy_air:.1f}%")
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values
    missing_values = cleaned_df_aq.isnull().sum().sum()
    if missing_values > 0:
        print(f"Missing values detected: {missing_values}")
        quality_issues += 1
    
    # check for negative values (i.e., should not exist)
    negative_counts = (cleaned_df_aq['count'] < 0).sum()
    negative_percentages = (cleaned_df_aq['percentage'] < 0).sum()
    
    if negative_counts > 0:
        print(f"Negative count values: {negative_counts}")
        quality_issues += 1
    if negative_percentages > 0:
        print(f"Negative percentage values: {negative_percentages}")
        quality_issues += 1
    
    # check percentage sum
    percentage_sum = cleaned_df_aq['percentage'].sum()
    if abs(percentage_sum - 100) > 0.1:
        print(f"Percentage sum deviation: {percentage_sum:.1f}% (should be ~100%)")
        quality_issues += 1
    
    # check for duplicate bins
    duplicates = cleaned_df_aq['bin'].duplicated().sum()
    if duplicates > 0:
        print(f"Duplicate concentration bins: {duplicates}")
        quality_issues += 1
    
    # check for expected number of bins (should be 9)
    if len(cleaned_df_aq) != 9:
        print(f"Unexpected number of bins: {len(cleaned_df_aq)} (expected 9)")
        quality_issues += 1
    
    if quality_issues == 0:
        print("No data quality issues detected")
    
except Exception as e:
    print(f"Error processing air quality data: {e}")
    print("Troubleshooting steps:")
    print("   1. Ensure TIF file exists at the specified path")
    print("   2. Check that the TIF file contains PM2.5 concentration values")
    print("   3. Verify the TIF file is not corrupted")
    print("   4. Ensure rasterio library is installed: pip install rasterio")
    print("   5. Check file permissions and disk space")
    print("   6. Verify concentration values are realistic (0-500+ μg/m³)")

print("\n" + "="*60)

AIR QUALITY (PM2.5) DATA PROCESSING
Analyzing TIF data structure...
PM2.5 concentration range: 55.30 - 61.20
NoData value: -3.4028234663852886e+38
Total valid pixels: 290
Unique concentration values: 58
Mean concentration: 58.02 μg/m³
Median concentration: 57.80 μg/m³
Standard deviation: 1.25 μg/m³

----------------------------------------
PM2.5 data range: 55.30 - 61.20 μg/m³
Unique values in data: 58
Cleaned air quality data saved to: data/processed/aq_area.csv
PM2.5 concentration bins: 9
Total pixels analyzed: 290
Percentage coverage verification: 100.0% (should be ~100%)
Most common PM2.5 range: 50-100 μg/m³ (100.0%)
Air quality data processed successfully!

Cleaned data shape: (9, 3)
Cleaned data columns: ['bin', 'count', 'percentage']

Processed Air Quality data:
      bin  count  percentage
0     0-5      0         0.0
1    5-10      0         0.0
2   10-15      0         0.0
3   15-20      0         0.0
4   20-30      0         0.0
5   30-40      0         0.0
6   40-50      0 

### SUMMER SURFACE TEMPERATURE

### summer_area.csv preparation (i.e., % area with different summer surface temperatures in degrees Celsius (°C))
### Observable Notebook functions/charts:
#### 1.) "plot_summer_area" / "chart_summer_area" (i.e., Percentage of area with different summer surface temperatures in degrees Celsius (°C))

In [ ]:
# SUMMER SURFACE TEMPERATURE - summer_area.csv preparation from raw tif data for Observable Notebook plot functions/charts:
# 1.) "plot_summer_area" / "chart_summer_area" (i.e., Percentage of Area with different summer surface temperatures in degrees Celsius (°C))

# load "raw" (i.e. "dirty") tif data
input_tif_path = 'data/raw/2025-04-colombia-cartagena_02-process-output_spatial_cartagena_summer.tif' # updatefile path

print("="*60)
print("SUMMER SURFACE TEMPERATURE DATA PROCESSING")
print("="*60)

print("Analyzing TIF data structure...")

try:
    import rasterio
    with rasterio.open(input_tif_path) as src:
        data = src.read(1)
        # remove "NaN" and "NoData" values for analysis
        if src.nodata is not None:
            valid_data = data[data != src.nodata]
        else:
            valid_data = data[~np.isnan(data)]
        
        # remove infinite values
        valid_data = valid_data[np.isfinite(valid_data)]
        
        print(f"Temperature range: {valid_data.min():.1f}°C - {valid_data.max():.1f}°C")
        print(f"NoData value: {src.nodata}")
        print(f"Total valid pixels: {len(valid_data):,}")
        print(f"Unique temperature values: {len(np.unique(valid_data)):,}")
        
        # show basic statistics
        print(f"Mean temperature: {valid_data.mean():.1f}°C")
        print(f"Median temperature: {np.median(valid_data):.1f}°C")
        print(f"Standard deviation: {valid_data.std():.1f}°C")

except Exception as e:
    print(f"Could not examine TIF structure: {e}")

print("\n" + "-"*40)

# process the tif file using clean_summer_area function in clean.py
# bin_width=5 creates 5°C temperature bins (default)
try:
    cleaned_df_summer = clean_summer_area(input_tif_path, bin_width=5)
    print("Summer temperature data processed successfully!")
    
    # display cleaned data structure
    print(f"\nCleaned data shape: {cleaned_df_summer.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_summer.columns)}")
    print(f"\nProcessed Summer Temperature data:")
    print(cleaned_df_summer)
    
    # basic data validation
    print(f"\nData Validation:")
    print(f"- Missing values: {cleaned_df_summer.isnull().sum().sum()}")
    print(f"- Temperature bins: {len(cleaned_df_summer)}")
    print(f"- Total pixels: {cleaned_df_summer['count'].sum():,.0f}")
    print(f"- Percentage sum: {cleaned_df_summer['percentage'].sum():.1f}% (should be ~100%)")
    
    # temperature distribution analysis
    print(f"\nTemperature Distribution:")
    
    total_pixels = cleaned_df_summer['count'].sum()
    
    for idx, row in cleaned_df_summer.iterrows():
        if row['count'] > 0:  # only show bins with data
            print(f"- {row['bin']}°C: {row['count']:,.0f} pixels ({row['percentage']:.1f}%)")
    
    # temperature range analysis
    if len(cleaned_df_summer) > 0:
        # filter out zero-count bins
        active_bins = cleaned_df_summer[cleaned_df_summer['count'] > 0]
        
        if len(active_bins) > 0:
            max_temp_bin = active_bins.loc[active_bins['percentage'].idxmax()]
            min_temp_bin = active_bins.loc[active_bins['percentage'].idxmin()]
            
            print(f"\n- Most common temperature range: {max_temp_bin['bin']}°C ({max_temp_bin['percentage']:.1f}%)")
            print(f"- Least common temperature range: {min_temp_bin['bin']}°C ({min_temp_bin['percentage']:.1f}%)")
    
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values
    missing_values = cleaned_df_summer.isnull().sum().sum()
    if missing_values > 0:
        print(f"Missing values detected: {missing_values}")
        quality_issues += 1
    
    # check for negative values in "count"/"percentage"
    negative_counts = (cleaned_df_summer['count'] < 0).sum()
    negative_percentages = (cleaned_df_summer['percentage'] < 0).sum()
    
    if negative_counts > 0:
        print(f"Negative count values: {negative_counts}")
        quality_issues += 1
    if negative_percentages > 0:
        print(f"Negative percentage values: {negative_percentages}")
        quality_issues += 1
    
    # check percentage sum
    percentage_sum = cleaned_df_summer['percentage'].sum()
    if abs(percentage_sum - 100) > 0.1:
        print(f"Percentage sum deviation: {percentage_sum:.1f}% (should be ~100%)")
        quality_issues += 1
    
    # check for duplicate bins
    duplicates = cleaned_df_summer['bin'].duplicated().sum()
    if duplicates > 0:
        print(f"Duplicate temperature bins: {duplicates}")
        quality_issues += 1
    
    if quality_issues == 0:
        print("No data quality issues detected")
      
except Exception as e:
    print(f"Error processing summer temperature data: {e}")
    print("Troubleshooting steps:")
    print("   1. Ensure TIF file exists at the specified path")
    print("   2. Check that the TIF file contains temperature data in Celsius")
    print("   3. Verify the TIF file is not corrupted")
    print("   4. Ensure rasterio library is installed: pip install rasterio")
    print("   5. Check file permissions and disk space")

print("\n" + "="*60)

### Green Space, NDVI (Normalized Difference Vegetated Index)

### ndvi_area.csv preparation (i.e., percentage area with different NDVI - - i.e., "Water", [-1-0.015); "Built-up", [0.015-0.14); "Barren", [0.14-0.18); "Shrub and Grassland", [0.18-0.27); "Sparse", [0.27-0.36); and "Dense",   [0.36-1])
### Observable Notebook functions/charts:
#### 1.) "plot_ndvi_area" / "chart_ndvi_area" (i.e., percentage area with different NDVI - - i.e., "Water", [-1-0.015); "Built-up", [0.015-0.14); "Barren", [0.14-0.18); "Shrub and Grassland", [0.18-0.27); "Sparse", [0.27-0.36); and "Dense",   [0.36-1])

In [ ]:
# GREEN SPACE, NDVI (Normalized Difference Vegetated Index) - ndvi_area.csv preparation from raw tif data for Observable Notebook plot functions/charts:
# 1.) "plot_ndvi_area" / "chart_ndvi_area" (i.e., percentage area with different NDVI - - i.e., "Water", [-1-0.015); "Built-up", [0.015-0.14); "Barren", [0.14-0.18); "Shrub and Grassland", [0.18-0.27); "Sparse", [0.27-0.36); and "Dense",   [0.36-1])

# load "raw" (i.e. "dirty") tif data
input_tif_path = 'data/raw/2025-04-colombia-cartagena_02-process-output_spatial_cartagena_ndvi_season.tif' # updatefile path

print("="*60)
print("NDVI GREEN SPACE DATA PROCESSING")
print("="*60)

print("Analyzing TIF data structure...")

try:
    import rasterio
    with rasterio.open(input_tif_path) as src:
        data = src.read(1)
        # remove "NaN" and "NoData" values for analysis
        if src.nodata is not None:
            valid_data = data[data != src.nodata]
        else:
            valid_data = data[~np.isnan(data)]
        
        # remove infinite values
        valid_data = valid_data[np.isfinite(valid_data)]
        
        print(f"NDVI range: {valid_data.min():.3f} - {valid_data.max():.3f}")
        print(f"NoData value: {src.nodata}")
        print(f"Total valid pixels: {len(valid_data):,}")
        print(f"Unique NDVI values: {len(np.unique(valid_data)):,}")
        
        # show some basic statistics
        print(f"Mean NDVI: {valid_data.mean():.3f}")
        print(f"Median NDVI: {np.median(valid_data):.3f}")
        print(f"Standard deviation: {valid_data.std():.3f}")

except Exception as e:
    print(f"Could not examine TIF structure: {e}")

print("\n" + "-"*40)

# process the tif file using "clean_ndvi_area" function in clean.py
try:
    cleaned_df_ndvi = clean_ndvi_area(input_tif_path)
    print("NDVI data processed successfully!")
    
    # cleaned data structure
    print(f"\nCleaned data shape: {cleaned_df_ndvi.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_ndvi.columns)}")
    print(f"\nProcessed NDVI data:")
    print(cleaned_df_ndvi)
    
    # basic data validation
    print(f"\nData Validation:")
    print(f"- Missing values: {cleaned_df_ndvi.isnull().sum().sum()}")
    print(f"- Vegetation categories: {len(cleaned_df_ndvi)}")
    print(f"- Total pixels: {cleaned_df_ndvi['count'].sum():,.0f}")
    print(f"- Percentage sum: {cleaned_df_ndvi['percentage'].sum():.1f}% (should be ~100%)")
    
    # NDVI vegetation distribution analysis
    print(f"\nVegetation Cover Distribution:")
    
    total_pixels = cleaned_df_ndvi['count'].sum()
    
    for idx, row in cleaned_df_ndvi.iterrows():
        if row['count'] > 0:  # only show categories with data
            print(f"- {row['type']} {row['bin']}: {row['count']:,.0f} pixels ({row['percentage']:.1f}%)")
    
    # vegetation type analysis
    if len(cleaned_df_ndvi) > 0:
        # filter out zero-count categories
        active_categories = cleaned_df_ndvi[cleaned_df_ndvi['count'] > 0]
        
        if len(active_categories) > 0:
            max_vegetation = active_categories.loc[active_categories['percentage'].idxmax()]
            min_vegetation = active_categories.loc[active_categories['percentage'].idxmin()]
            
            print(f"\n- Most common cover: {max_vegetation['type']} ({max_vegetation['percentage']:.1f}%)")
            print(f"- Least common cover: {min_vegetation['type']} ({min_vegetation['percentage']:.1f}%)")
    
    # green space analysis
    dense_vegetation = cleaned_df_ndvi[cleaned_df_ndvi['type'] == 'Dense']['percentage'].sum()
    sparse_vegetation = cleaned_df_ndvi[cleaned_df_ndvi['type'] == 'Sparse']['percentage'].sum()
    shrub_grassland_vegetation = cleaned_df_ndvi[cleaned_df_ndvi['type'] == 'Shrub and Grassland']['percentage'].sum()
    total_green_space = dense_vegetation + sparse_vegetation + shrub_grassland_vegetation

    # non green space analysis
    barren_space = cleaned_df_ndvi[cleaned_df_ndvi['type'] == 'Barren']['percentage'].sum()
    built_up_space = cleaned_df_ndvi[cleaned_df_ndvi['type'] == 'Built-up']['percentage'].sum()
    water_space = cleaned_df_ndvi[cleaned_df_ndvi['type'] == 'Water']['percentage'].sum()
    total_non_green_space = barren_space + built_up_space + water_space

    
    print(f"\nGreen Space Analysis:")
    print(f"- Dense vegetation coverage: {dense_vegetation:.1f}%")
    print(f"- Sparse vegetation coverage: {sparse_vegetation:.1f}%")
    print(f"- Shrub and Grassland vegetation coverage: {shrub_grassland_vegetation:.1f}%")
    print(f"- Total green space coverage: {total_green_space:.1f}%")

    print(f"\nNon Green Space Analysis:")
    print(f"- Barren coverage: {barren_space:.1f}%")
    print(f"- Built-up coverage: {built_up_space:.1f}%")
    print(f"- Water coverage: {water_space:.1f}%")
    print(f"- Total non green space coverage: {total_non_green_space:.1f}%")
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values
    missing_values = cleaned_df_ndvi.isnull().sum().sum()
    if missing_values > 0:
        print(f"Missing values detected: {missing_values}")
        quality_issues += 1
    
    # check for negative values in "count"/"percentage"
    negative_counts = (cleaned_df_ndvi['count'] < 0).sum()
    negative_percentages = (cleaned_df_ndvi['percentage'] < 0).sum()
    
    if negative_counts > 0:
        print(f"Negative count values: {negative_counts}")
        quality_issues += 1
    if negative_percentages > 0:
        print(f"Negative percentage values: {negative_percentages}")
        quality_issues += 1
    
    # check percentage sum
    percentage_sum = cleaned_df_ndvi['percentage'].sum()
    if abs(percentage_sum - 100) > 0.1:
        print(f"Percentage sum deviation: {percentage_sum:.1f}% (should be ~100%)")
        quality_issues += 1
    
    # check for duplicate categories
    duplicates_bin = cleaned_df_ndvi['bin'].duplicated().sum()
    duplicates_type = cleaned_df_ndvi['type'].duplicated().sum()
    
    if duplicates_bin > 0:
        print(f"Duplicate NDVI bins: {duplicates_bin}")
        quality_issues += 1
    if duplicates_type > 0:
        print(f"Duplicate vegetation types: {duplicates_type}")
        quality_issues += 1
    
    # check for expected number of categories (i.e., "type"s) (should be 6)
    if len(cleaned_df_ndvi) != 6:
        print(f"Unexpected number of categories: {len(cleaned_df_ndvi)} (expected 6)")
        quality_issues += 1
    
    # check for expected vegetation types
    expected_types = ['Water', 'Built-up', 'Barren', 'Shrub and Grassland', 'Sparse', 'Dense']
    actual_types = cleaned_df_ndvi['type'].tolist()
    missing_types = set(expected_types) - set(actual_types)
    if missing_types:
        print(f"Categories not present in data: {missing_types}")
    
    if quality_issues == 0:
        print("No data quality issues detected")
    
    
except Exception as e:
    print(f"Error processing NDVI data: {e}")
    print("Troubleshooting steps:")
    print("   1. Ensure TIF file exists at the specified path")
    print("   2. Check that the TIF file contains NDVI values (-1 to 1 range)")
    print("   3. Verify the TIF file is not corrupted")
    print("   4. Ensure rasterio library is installed: pip install rasterio")
    print("   5. Check file permissions and disk space")
    print("   6. Verify NDVI values are realistic (-1.0 to 1.0 range)")

print("\n" + "="*60)

### Forests and Deforestation

### deforestation_area.csv preparation (i.e., percentage area that is forested and deforested (by year)
### Observable Notebook functions/charts:
#### 1.) "plot_deforestation_area1" / "chart_deforestation_area1" (i.e., cumulative forest loss over time, 2000-2023)
#### 2.) "plot_deforestation_area2" / "chart_deforestation_area2" (i.e., cumulative forest remaining over time, 2000-2023)
#### 3.) "plot_deforestation_area3" / "chart_deforestation_area3" (i.e., annual deforestation as percentage of original forest)
#### 4.) "plot_deforestation_area4" / "chart_deforestation_area4" (i.e., annual deforestation as percentage of total deforestation)
#### 5.) "plot_deforestation_area5" / "chart_deforestation_area5" (i.e., year-by-year forest composition)
#### 6.) "plot_deforestation_area6" / "chart_deforestation_area6" (i.e., progression of deforestation over time)
....

In [8]:
# FORESTS AND DEFORESTATION - deforestation_area.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_deforestation_area1" / "chart_deforestation_area1" (i.e., cumulative forest loss over time, 2000-2023)
# 2.) "plot_deforestation_area2" / "chart_deforestation_area2" (i.e., cumulative forest remaining over time, 2000-2023)
#3.) "plot_deforestation_area3" / "chart_deforestation_area3" (i.e., annual deforestation as percentage of original forest)
# 4.) "plot_deforestation_area4" / "chart_deforestation_area4" (i.e., annual deforestation as percentage of total deforestation)
#5.) "plot_deforestation_area5" / "chart_deforestation_area5" (i.e., year-by-year forest composition)
#6.) "plot_deforestation_area6" / "chart_deforestation_area6" (i.e., progression of deforestation over time)

# load "raw" (i.e. "dirty") tif data
forest_tif_path = 'data/raw/2025-11-mauritania-nouakchott_02-process-output_spatial_nouakchott_forest_cover23.tif' # updatefile path
deforestation_tif_path = 'data/raw/2025-11-mauritania-nouakchott_02-process-output_spatial_nouakchott_deforestation.tif' # updatefile path

print("="*60)
print("FOREST COVER & DEFORESTATION DATA PROCESSING")
print("="*60)

# process tif files using "clean_deforestation_area" function in clean.py
# "base_year=2000" means that value "1" = 2001; and  value "23" = 2023
# note: "auto_align=True" will automatically fix any alignment issues
try:
    cleaned_df_deforest = clean_deforestation_area(
        forest_tif_file=forest_tif_path,
        deforestation_tif_file=deforestation_tif_path,
        base_year=2000,
        auto_align=True
    )
    print("\nDeforestation data processed successfully!")
    
    # cleaned data structure
    print(f"\nYear-over-Year Deforestation Data:")
    print(cleaned_df_deforest.to_string(index=False))
    
    # data validation
    print(f"\n" + "="*60)
    print("DATA VALIDATION:")
    print(f"- Total years tracked: {len(cleaned_df_deforest)}")
    print(f"- Missing values: {cleaned_df_deforest.isnull().sum().sum()}")
    
    # check that percentages add up correctly (should be 100%)
    final_row = cleaned_df_deforest.iloc[-1]
    percent_check = final_row['percent_forest_remaining'] + final_row['percent_forest_lost']
    print(f"- Percent remaining + lost: {percent_check:.1f}% (should be 100%)")
    
    # deforestation trend analysis
    print(f"\n" + "="*60)
    print("DEFORESTATION TRENDS:")
    
    years_with_deforest = cleaned_df_deforest[cleaned_df_deforest['deforested_this_year'] > 0]
    
    if len(years_with_deforest) > 0:
        # overall statistics
        total_deforested = final_row['cumulative_deforested']
        avg_annual = years_with_deforest['deforested_this_year'].mean()
        
        print(f"- Years with deforestation: {len(years_with_deforest)}")
        print(f"- Average annual loss: {avg_annual:.0f} pixels/year")
        print(f"- Total forest lost: {total_deforested:,} pixels ({final_row['percent_forest_lost']:.2f}%)")
        
        # peak year of deforestation
        peak = years_with_deforest.loc[years_with_deforest['deforested_this_year'].idxmax()]
        print(f"- Peak deforestation: {peak['year']} ({peak['deforested_this_year']:,} pixels)")
        
        # recent deforestation trend (i.e., last 5 years)
        recent_5 = years_with_deforest.tail(5)
        recent_total = recent_5['deforested_this_year'].sum()
        recent_avg = recent_5['deforested_this_year'].mean()
        
        print(f"\nRecent 5 Years (last available data):")
        print(f"- Total deforested: {recent_total:,} pixels")
        print(f"- Average annual: {recent_avg:.0f} pixels/year")
        
        # compare recent deforestation vs overall average deforestation
        # note:
        # ratio = recent 5 years (pixel/year) / overall historical average (pixels/year), example: 13(pixel/year) / 18(pixel/year) = 0.7x overall average - SLOWING!

        # ratio is > 1, recent deforestaion is FASTER than historic average deforestation (i.e., accelerating);
        # ratio is < 1, recent deforestaion is SLOWER than historic average deforestation (i.e., decelerating);
        # ratio is = 1, recent deforestaion is at the SAME rate as historic average deforestation (i.e., steady);
        if recent_avg > avg_annual:
            print(f"- Trend: Accelerating ({recent_avg/avg_annual:.1f}x overall average)")
        else:
            print(f"- Trend: Slowing ({recent_avg/avg_annual:.1f}x overall average)")
    
    # data quality checks
    print(f"\n" + "="*60)
    print("DATA QUALITY CHECKS:")
    
    quality_issues = 0
    
    # check for negative values
    if (cleaned_df_deforest['forest_remaining'] < 0).any():
        print("Negative forest remaining values detected")
        quality_issues += 1
    
    # check that cumulative never decreases
    cumulative_diff = cleaned_df_deforest['cumulative_deforested'].diff()
    if (cumulative_diff[cumulative_diff.notna()] < 0).any():
        print("Cumulative deforestation decreases (impossible)")
        quality_issues += 1
    
    # check that forest remaining + cumulative = baseline
    baseline = cleaned_df_deforest.iloc[0]['forest_remaining']
    for idx, row in cleaned_df_deforest.iterrows():
        if row['forest_remaining'] + row['cumulative_deforested'] != baseline:
            print(f"Year {row['year']}: Forest accounting error")
            quality_issues += 1
            break
    
    if quality_issues == 0:
        print("No data quality issues detected")
    
except Exception as e:
    print(f"Error processing deforestation data: {e}")
    print("\nTroubleshooting:")
    print("1. Check file paths are correct")
    print("2. Ensure rasterio is installed: pip install rasterio")
    print("3. Verify TIF files are valid and not corrupted")

print("\n" + "="*60)

FOREST COVER & DEFORESTATION DATA PROCESSING
Alignment check:
- Same CRS: True
- Same shape: True
- Same bounds: True
- Fully aligned: True
TIFs are properly aligned

Baseline forest area: 0 pixels

Cleaned deforestation data saved to: data/processed/deforestation_area.csv
Time period: 2000 - 2000.0
Baseline forest: 0 pixels
Total deforested: 0.0 pixels (0.00%)
Forest remaining: 0.0 pixels (100.00%)
Error processing deforestation data: 0

Troubleshooting:
1. Check file paths are correct
2. Ensure rasterio is installed: pip install rasterio
3. Verify TIF files are valid and not corrupted



# RISK IDENTIFICATION

### FLOOD EVENTS

### fu.csv, pu.csv, cu.csv, and comb.csv preparation
### Observable Notebook functions/charts:
#### 1.) "plot_fu" / "chart_fu" (i.e., built-up area exposed to river (fluvial) flooding)
#### 2.) "plot_pu" / "chart_pu" (i.e., built-up area exposed to rainwater (pluvial) flooding)
#### 3.) "plot_cu" / "chart_cu" (i.e., built-up area exposed to coastal flooding)
#### 4.) "plot_comb" / "chart_comb" (i.e., built-up area exposed to combined flooding)

#### NOTE:
#### 5.) data, raw, "flood-events.csv" is already ready for Observable Notebook "plot_fe" / "chart_fe" (i.e.,large flood events in city, country)

In [9]:
# FLOODING - multiple csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_fu"  / "chart_fu" (i.e., built-up area exposed to river (fluvial) flooding)
# 2.) "plot_pu"  / "chart_pu" (i.e., built-up area exposed to rainwater (pluvial) flooding)
# 3.) "plot_cu"  / "chart_cu" (i.e., built-up area exposed to coastal flooding)
# 4.) "plot_comb" / "chart_comb" (i.e., built-up area exposed to combined flooding)
#5.) Note: data, raw, "flood-events.csv" is already ready for Observable Notebook "plot_fe" / "chart_fe" (i.e.,large flood events in city, country)


# load "raw" (i.e. "dirty") tabular output data
# Note: flood data structure may vary by city - (i.e., coastal cities have coastal flood data, inland cities do not)
raw_df_flood = pd.read_csv('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_flood_wsf.csv') # updatefile path

# basic info about raw data
print("Raw flood risk data info:")
print(f"Shape: {raw_df_flood.shape}")
print(f"Columns: {list(raw_df_flood.columns)}")
print(f"Year range: {raw_df_flood['year'].min()} - {raw_df_flood['year'].max()}")
print(f"Total data points: {len(raw_df_flood)}")

# ID available flood types
flood_columns = [col for col in raw_df_flood.columns if '_2020' in col]
print(f"Available flood types: {flood_columns}")

# preview flood risk ranges for each type
for col in flood_columns:
    flood_type = col.replace('_2020', '')
    print(f"{flood_type.capitalize()} risk range: {raw_df_flood[col].min():.2f} - {raw_df_flood[col].max():.2f}")

print(f"Data preview:")
print(raw_df_flood.head())
print("\n" + "="*50 + "\n")

# clean data using clean_flood function in clean.py
try:
    created_files = clean_flood('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_flood_wsf.csv') # updatefile path
    print("Flood risk data processed successfully!")
    
    # load and validate each created file
    flood_dataframes = {}
    
    for filename in created_files:
        file_path = f'data/processed/{filename}'
        flood_type = filename.replace('.csv', '')
        
        try:
            df = pd.read_csv(file_path)
            flood_dataframes[flood_type] = df
            
            print(f"\n{filename} validation:")
            print(f"- Shape: {df.shape}")
            print(f"- Columns: {list(df.columns)}")
            print(f"- Year range: {df['yearName'].min()} - {df['yearName'].max()}")
            print(f"- Risk range: {df.iloc[:, 2].min():.2f} - {df.iloc[:, 2].max():.2f}")  # Note: third column is risk value
            print(f"- Sample data:")
            print(df.head(5))
            
        except Exception as e:
            print(f"Error loading {filename}: {e}")
    
    # basic flood risk analysis
    if len(flood_dataframes) > 0:
        print(f"\nFlood Risk Data Summary:")
        
        for flood_type, df in flood_dataframes.items():
            risk_column = df.columns[2]  # Note: third column contains risk values
            
            # basic statistics
            avg_risk = df[risk_column].mean()
            max_risk = df[risk_column].max()
            min_risk = df[risk_column].min()
            std_risk = df[risk_column].std()
            
            # trend calculation
            trend = df[risk_column].iloc[-1] - df[risk_column].iloc[0]
            
            print(f"\n- {flood_type.upper()} flood risk:")
            print(f"  Average: {avg_risk:.2f}")
            print(f"  Range: {min_risk:.2f} - {max_risk:.2f}")
            print(f"  Standard deviation: {std_risk:.2f}")
            print(f"  Change (1985-2015): {trend:+.2f}")
        
        # current values
        if len(flood_dataframes) > 1:
            print(f"\n2015 Risk Values:")
            for flood_type, df in flood_dataframes.items():
                risk_column = df.columns[2]
                current_risk = df[risk_column].iloc[-1]
                print(f"- {flood_type.upper()}: {current_risk:.2f}")
        
        # data quality checks
        print(f"\nData Quality Checks:")
        
        quality_issues = 0
        for flood_type, df in flood_dataframes.items():
            risk_column = df.columns[2]
            
            # check for missing values
            missing_values = df[risk_column].isna().sum()
            if missing_values > 0:
                print(f"{flood_type.upper()}: {missing_values} missing values")
                quality_issues += 1
            
            # check for negative values (mathematically impossible for risk)
            negative_values = (df[risk_column] < 0).sum()
            if negative_values > 0:
                print(f"{flood_type.upper()}: {negative_values} negative risk values")
                quality_issues += 1
        
        if quality_issues == 0:
            print("No data quality issues detected")
    
except Exception as e:
    print(f"Error processing flood risk data: {e}")
    print("Check that the flood CSV file exists and has the correct format")
    print("Expected columns: year, coastal_2020, fluvial_2020, pluvial_2020, comb_2020 (as available)")

# save confirmation and next steps
if 'created_files' in locals() and len(created_files) > 0:
    print(f"\n Cleaned data saved to data/processed/:")
    for filename in created_files:
        print(f"    {filename}")
    
    # data structure summary
    print(f"\n Data structure summary:")
    print(f"- Files created: {len(created_files)}")
    print(f"- Time series length: {len(list(flood_dataframes.values())[0]) if flood_dataframes else 'N/A'} years")
    print(f"- Year range: {raw_df_flood['year'].min()} - {raw_df_flood['year'].max()}")
    print(f"- Data types: {dict(list(flood_dataframes.values())[0].dtypes) if flood_dataframes else 'N/A'}")
    
else:
    print("No cleaned flood data available")
    print("Troubleshooting steps:")
    print("   1. Ensure flood CSV file exists in data/raw/")
    print("   2. Check file has correct columns with '_2020' suffix")
    print("   3. Verify data covers expected time range (1985-2015)")
    print("   4. Check that at least one flood type column exists")

Raw flood risk data info:
Shape: (31, 5)
Columns: ['year', 'coastal_2020', 'comb_2020', 'fluvial_2020', 'pluvial_2020']
Year range: 1985 - 2015
Total data points: 31
Available flood types: ['coastal_2020', 'comb_2020', 'fluvial_2020', 'pluvial_2020']
Coastal risk range: 0.00 - 0.11
Comb risk range: 9.16 - 39.56
Fluvial risk range: 0.00 - 0.00
Pluvial risk range: 9.16 - 39.45
Data preview:
   year  coastal_2020  comb_2020  fluvial_2020  pluvial_2020
0  1985      0.000000   9.155106           0.0      9.155106
1  1986      0.000000   9.155106           0.0      9.155106
2  1987      0.000861  11.003510           0.0     11.002650
3  1988      0.001721  12.592896           0.0     12.591175
4  1989      0.002582  13.657364           0.0     13.654782


Available flood types: ['coastal', 'fluvial', 'pluvial', 'combined']
Created cu.csv: 31 records
   Year range: 1985 - 2015
   CU range: 0.00 - 0.11
Created fu.csv: 31 records
   Year range: 1985 - 2015
   FU range: 0.00 - 0.00
Created pu.cs

### ELEVATION

### e.csv preparation
### Observable Notebook functions/charts:
#### 1.) "plot_e" / "plot_e_alt" / "chart_e" (i.e., elevation)

In [2]:
# ELEVATION - e.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_e" / "chart_e" (elevation distribution vertical bar chart)

# load "raw" (i.e. "dirty") tabular output data
raw_df_e = pd.read_csv('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_elevation.csv') # updatefile path

# basic info about raw data
print("Raw elevation data info:")
print(f"Shape: {raw_df_e.shape}")
print(f"Columns: {list(raw_df_e.columns)}")
print(f"Total data points: {len(raw_df_e)}")

# preview elevation ranges and counts if available
if 'Bin' in raw_df_e.columns and 'Count' in raw_df_e.columns:
    print(f"Elevation bins: {len(raw_df_e)}")
    print(f"Total pixels: {raw_df_e['Count'].sum():,.0f}")
    print(f"Pixel count range: {raw_df_e['Count'].min():,.0f} - {raw_df_e['Count'].max():,.0f}")
    print(f"Elevation ranges: {raw_df_e['Bin'].iloc[0]} to {raw_df_e['Bin'].iloc[-1]}")

print(f"Data preview:")
print(raw_df_e.head())
print("\n" + "="*50 + "\n")

# clean data using clean_e function in clean.py
try:
    cleaned_df_e = clean_e('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_elevation.csv') # updatefile path
    print("Elevation data cleaned successfully!")
    
    # cleaned data info
    print(f"\nCleaned data shape: {cleaned_df_e.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_e.columns)}")
    print(f"Sample of cleaned data:")
    print(cleaned_df_e.head(10))
    
    # basic data validation
    print(f"\nData validation:")
    print(f"- Missing values: {cleaned_df_e.isnull().sum().sum()}")
    print(f"- Elevation bins: {len(cleaned_df_e)}")
    print(f"- Total pixels: {cleaned_df_e['count'].sum():,.0f}")
    print(f"- Percentage sum: {cleaned_df_e['percentage'].sum():.1f}% (should be ~100%)")
    
    # elevation data analysis
    print(f"\nElevation Data Summary:")
    
    # basic statistics
    avg_percentage = cleaned_df_e['percentage'].mean()
    median_percentage = cleaned_df_e['percentage'].median()
    
    print(f"- Elevation ranges present: {len(cleaned_df_e)}")
    print(f"- Pixel count range: {cleaned_df_e['count'].min():,.0f} - {cleaned_df_e['count'].max():,.0f}")
    print(f"- Percentage range: {cleaned_df_e['percentage'].min():.2f}% - {cleaned_df_e['percentage'].max():.2f}%")
    
    # ID extremes
    dominant_bin = cleaned_df_e.loc[cleaned_df_e['percentage'].idxmax()]
    least_common_bin = cleaned_df_e.loc[cleaned_df_e['percentage'].idxmin()]
    
    print(f"- Most common elevation: {dominant_bin['bin']} ({dominant_bin['percentage']:.2f}%)")
    print(f"- Least common elevation: {least_common_bin['bin']} ({least_common_bin['percentage']:.2f}%)")
    print(f"- Lowest elevation range: {cleaned_df_e['bin'].iloc[0]}")
    print(f"- Highest elevation range: {cleaned_df_e['bin'].iloc[-1]}")
    
    # coverage distribution
    above_20_percent = (cleaned_df_e['percentage'] >= 20).sum()
    above_10_percent = (cleaned_df_e['percentage'] >= 10).sum()
    above_5_percent = (cleaned_df_e['percentage'] >= 5).sum()
    
    print(f"- Ranges with ≥20% coverage: {above_20_percent}")
    print(f"- Ranges with ≥10% coverage: {above_10_percent}")
    print(f"- Ranges with ≥5% coverage: {above_5_percent}")
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values in key columns
    missing_bin = cleaned_df_e['bin'].isna().sum()
    missing_count = cleaned_df_e['count'].isna().sum()
    missing_percentage = cleaned_df_e['percentage'].isna().sum()
    
    if missing_bin > 0:
        print(f"Missing elevation bin values: {missing_bin}")
        quality_issues += 1
    if missing_count > 0:
        print(f"Missing count values: {missing_count}")
        quality_issues += 1
    if missing_percentage > 0:
        print(f"Missing percentage values: {missing_percentage}")
        quality_issues += 1
    
    # check for impossible values
    negative_count = (cleaned_df_e['count'] < 0).sum()
    negative_percentage = (cleaned_df_e['percentage'] < 0).sum()
    
    if negative_count > 0:
        print(f"Negative count values: {negative_count}")
        quality_issues += 1
    if negative_percentage > 0:
        print(f"Negative percentage values: {negative_percentage}")
        quality_issues += 1
    
    # check percentage sum
    percentage_sum = cleaned_df_e['percentage'].sum()
    if abs(percentage_sum - 100) > 0.1:
        print(f"Percentage sum deviation: {percentage_sum:.1f}% (should be ~100%)")
        quality_issues += 1
    
    # check for duplicate elevation bins
    duplicates = cleaned_df_e['bin'].duplicated().sum()
    if duplicates > 0:
        print(f"Duplicate elevation bins: {duplicates}")
        quality_issues += 1
    
    if quality_issues == 0:
        print("No data quality issues detected")
        
except Exception as e:
    print(f"Error cleaning elevation data: {e}")
    print("Check that the elevation CSV file exists and has the correct format")
    print("Expected columns: Bin, Count")

# save confirmation and next steps
if 'cleaned_df_e' in locals():
    print(f"\nCleaned data saved to: data/processed/e.csv")
    
    # preview of data structure
    print(f"\n Data structure summary:")
    print(f"- Columns: {list(cleaned_df_e.columns)}")
    print(f"- Data points: {len(cleaned_df_e)} elevation ranges")
    print(f"- Data types: {dict(cleaned_df_e.dtypes)}")
    print(f"- Elevation coverage: {cleaned_df_e['percentage'].min():.2f}% - {cleaned_df_e['percentage'].max():.2f}%")
    
else:
    print("No cleaned elevation data available")
    print("Troubleshooting steps:")
    print("   1. Ensure elevation CSV file exists in data/raw/")
    print("   2. Check file has correct columns: Bin, Count")
    print("   3. Verify elevation bins are properly labeled")
    print("   4. Check that count values are numeric and positive")

Raw elevation data info:
Shape: (5, 2)
Columns: ['Bin', 'Count']
Total data points: 5
Elevation bins: 5
Total pixels: 397,906
Pixel count range: 682 - 307,332
Elevation ranges: -6--1 to 14-19
Data preview:
     Bin   Count
0  -6--1    4166
1   -1-4  307332
2    4-9   73591
3   9-14   12135
4  14-19     682


Cleaned data saved to: data/processed/e.csv
Elevation bins: 5
Elevation range: 4-9 to -1-4
Total area analyzed: 397,906 pixels
Percentage coverage verification: 100.0% (should be ~100%)
Dominant elevation range: -1-4 (77.2%)
Major elevation ranges (≥10% coverage): 2 bins
Major ranges: 4-9, -1-4
Elevation data cleaned successfully!

Cleaned data shape: (5, 3)
Cleaned data columns: ['bin', 'count', 'percentage']
Sample of cleaned data:
     bin   count  percentage
0    4-9   73591       18.49
1   9-14   12135        3.05
2  14-19     682        0.17
3  -6--1    4166        1.05
4   -1-4  307332       77.24

Data validation:
- Missing values: 0
- Elevation bins: 5
- Total pixels: 397,

### SLOPE

### s.csv preparation
### Observable Notebook functions/charts:
#### 1.) "plot_s" / "chart_s" (i.e., slope)

In [12]:
# SLOPE - s.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_s" / "chart_s" (slope chart)

# load "raw" (i.e. "dirty") tabular output data
raw_df_s = pd.read_csv('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_slope.csv') # updatefile path

# basic info about raw data
print("Raw slope data info:")
print(f"Shape: {raw_df_s.shape}")
print(f"Columns: {list(raw_df_s.columns)}")
print(f"Total data points: {len(raw_df_s)}")

# slope ranges and counts
if 'Bin' in raw_df_s.columns and 'Count' in raw_df_s.columns:
    print(f"Slope bins: {len(raw_df_s)}")
    print(f"Total pixels: {raw_df_s['Count'].sum():,.0f}")
    print(f"Pixel count range: {raw_df_s['Count'].min():,.0f} - {raw_df_s['Count'].max():,.0f}")
    print(f"Slope ranges: {raw_df_s['Bin'].iloc[0]} to {raw_df_s['Bin'].iloc[-1]} degrees")

print(f"Data preview:")
print(raw_df_s.head())
print("\n" + "="*50 + "\n")

# clean data using clean_s function in clean.py
try:
    cleaned_df_s = clean_s('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_slope.csv') # updatefile path
    print("Slope data cleaned successfully!")
    
    # cleaned data info
    print(f"\nCleaned data shape: {cleaned_df_s.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_s.columns)}")
    print(f"Sample of cleaned data:")
    print(cleaned_df_s.head(10))
    
    # basic data validation
    print(f"\nData validation:")
    print(f"- Missing values: {cleaned_df_s.isnull().sum().sum()}")
    print(f"- Slope bins: {len(cleaned_df_s)}")
    print(f"- Total pixels: {cleaned_df_s['count'].sum():,.0f}")
    print(f"- Percentage sum: {cleaned_df_s['percentage'].sum():.1f}% (should be ~100%)")
    
    # slope data analysis
    print(f"\nSlope Data Summary:")
    
    print(f"- Slope ranges present: {len(cleaned_df_s)}")
    print(f"- Pixel count range: {cleaned_df_s['count'].min():,.0f} - {cleaned_df_s['count'].max():,.0f}")
    print(f"- Percentage range: {cleaned_df_s['percentage'].min():.2f}% - {cleaned_df_s['percentage'].max():.2f}%")
    
    # ID extremes
    dominant_bin = cleaned_df_s.loc[cleaned_df_s['percentage'].idxmax()]
    least_common_bin = cleaned_df_s.loc[cleaned_df_s['percentage'].idxmin()]
    
    print(f"- Most common slope: {dominant_bin['bin']} degrees ({dominant_bin['percentage']:.2f}%)")
    print(f"- Least common slope: {least_common_bin['bin']} degrees ({least_common_bin['percentage']:.2f}%)")
    print(f"- Gentlest slope range: {cleaned_df_s['bin'].iloc[0]} degrees")
    print(f"- Steepest slope range: {cleaned_df_s['bin'].iloc[-1]} degrees")
    
    # coverage distribution
    above_20_percent = (cleaned_df_s['percentage'] >= 20).sum()
    above_10_percent = (cleaned_df_s['percentage'] >= 10).sum()
    above_5_percent = (cleaned_df_s['percentage'] >= 5).sum()
    
    print(f"- Ranges with ≥20% coverage: {above_20_percent}")
    print(f"- Ranges with ≥10% coverage: {above_10_percent}")
    print(f"- Ranges with ≥5% coverage: {above_5_percent}")
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values in key columns
    missing_bin = cleaned_df_s['bin'].isna().sum()
    missing_count = cleaned_df_s['count'].isna().sum()
    missing_percentage = cleaned_df_s['percentage'].isna().sum()
    
    if missing_bin > 0:
        print(f"Missing slope bin values: {missing_bin}")
        quality_issues += 1
    if missing_count > 0:
        print(f"Missing count values: {missing_count}")
        quality_issues += 1
    if missing_percentage > 0:
        print(f"Missing percentage values: {missing_percentage}")
        quality_issues += 1
    
    # check for impossible values
    negative_count = (cleaned_df_s['count'] < 0).sum()
    negative_percentage = (cleaned_df_s['percentage'] < 0).sum()
    
    if negative_count > 0:
        print(f"Negative count values: {negative_count}")
        quality_issues += 1
    if negative_percentage > 0:
        print(f"Negative percentage values: {negative_percentage}")
        quality_issues += 1
    
    # check percentage sum
    percentage_sum = cleaned_df_s['percentage'].sum()
    if abs(percentage_sum - 100) > 0.1:
        print(f"Percentage sum deviation: {percentage_sum:.1f}% (should be ~100%)")
        quality_issues += 1
    
    # check for duplicate slope bins
    duplicates = cleaned_df_s['bin'].duplicated().sum()
    if duplicates > 0:
        print(f"Duplicate slope bins: {duplicates}")
        quality_issues += 1
    
    if quality_issues == 0:
        print("No data quality issues detected")
        
except Exception as e:
    print(f"Error cleaning slope data: {e}")
    print("Check that the slope CSV file exists and has the correct format")
    print("Expected columns: Bin, Count")

# save confirmation and next steps
if 'cleaned_df_s' in locals():
    print(f"\nCleaned data saved to: data/processed/s.csv")
    
    # quick preview of data structure
    print(f"\nData structure summary:")
    print(f"- Columns: {list(cleaned_df_s.columns)}")
    print(f"- Data points: {len(cleaned_df_s)} slope ranges")
    print(f"- Data types: {dict(cleaned_df_s.dtypes)}")
    print(f"- Slope coverage: {cleaned_df_s['percentage'].min():.2f}% - {cleaned_df_s['percentage'].max():.2f}%")
      
else:
    print("No cleaned slope data available")
    print("Troubleshooting steps:")
    print("   1. Ensure slope CSV file exists in data/raw/")
    print("   2. Check file has correct columns: Bin, Count")
    print("   3. Verify slope bins are properly labeled (e.g., 0-2, 2-5)")
    print("   4. Check that count values are numeric and positive")

Raw slope data info:
Shape: (5, 2)
Columns: ['Bin', 'Count']
Total data points: 5
Slope bins: 5
Total pixels: 397,907
Pixel count range: 0 - 388,800
Slope ranges: 0-2 to 20-90 degrees
Data preview:
     Bin   Count
0    0-2  388800
1    2-5    8791
2   5-10     308
3  10-20       8
4  20-90       0


Cleaned data saved to: data/processed/s.csv
Slope bins: 4
Slope range: 0-2 to 10-20 degrees
Total area analyzed: 397,907 pixels
Percentage coverage verification: 100.0% (should be ~100%)
Dominant slope range: 0-2 degrees (97.7%)
Relatively flat areas (0-5 degrees): 97.7%
Significant slope ranges (≥5% coverage): 1 bins
Significant ranges: 0-2
Slope data cleaned successfully!

Cleaned data shape: (4, 3)
Cleaned data columns: ['bin', 'count', 'percentage']
Sample of cleaned data:
     bin   count  percentage
0    0-2  388800       97.71
1    2-5    8791        2.21
2   5-10     308        0.08
3  10-20       8        0.00

Data validation:
- Missing values: 0
- Slope bins: 4
- Total pixels: 3

### LANDSLIDE SUSCEPTIBILITY

### ls_area.csv preparation (i.e., % area with different landslide susceptibility levels - "No Data" (0); "Very low" (1); "Low" (2); "Medium" (3); "High" (4); and "Very high" (5))
### Observable Notebook functions/charts:
#### 1.) "plot_ls_area" / "chart_ls_area" (i.e., Percentage of area with different landslide susceptibiltiy levels, "No Data" (0); "Very low" (1); "Low" (2); "Medium" (3); "High" (4); and "Very high" (5)

In [13]:
# LANDSLIDE SUSCEPTIBILITY - ls_area.csv preparation from raw tif data for Observable Notebook plot functions/charts:
# 1.) "plot_ls_area" / "chart_ls_area" (i.e., Percentage of Area with different landslide susceptibility levels, "No Data" (0); "Very low" (1); "Low" (2); "Medium" (3); "High" (4); and "Very high" (5)

# load "raw" (i.e. "dirty") tif data
input_tif_path = 'data/raw/2025-11-mauritania-nouakchott_02-process-output_spatial_nouakchott_landslide.tif'  # updatefile path

print("="*60)
print("LANDSLIDE SUSCEPTIBILITY DATA PROCESSING")
print("="*60)

# data value distribution
print("Analyzing TIF data structure...")

try:
    import rasterio
    with rasterio.open(input_tif_path) as src:
        data = src.read(1)
        unique_vals = np.unique(data[~np.isnan(data)])
        print(f"Unique values in TIF: {unique_vals}")
        print(f"Data range: {data.min()} to {data.max()}")
        print(f"NoData value: {src.nodata}")
        
        # count pixels for each value
        for val in unique_vals:
            count = np.sum(data == val)
            print(f"Value {val}: {count:,} pixels")

except Exception as e:
    print(f"Could not examine TIF structure: {e}")

print("\n" + "-"*40)

# process tif file using clean_ls_area function in clean.py
# set "include_nodata=False" to exclude value, "0"  ("NoData"/background)
try:
    cleaned_df_landslide = clean_ls_area(input_tif_path, include_nodata=False)
    print("Landslide susceptibility data processed successfully!")
    
    # cleaned data structure
    print(f"\nCleaned data shape: {cleaned_df_landslide.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_landslide.columns)}")
    print(f"\nProcessed Landslide Susceptibility data:")
    print(cleaned_df_landslide)
    
    # basic data validation
    print(f"\nData Validation:")
    print(f"- Missing values: {cleaned_df_landslide.isnull().sum().sum()}")
    print(f"- Susceptibility categories: {len(cleaned_df_landslide)}")
    print(f"- Total pixels: {cleaned_df_landslide['count'].sum():,.0f}")
    print(f"- Percentage sum: {cleaned_df_landslide['percentage'].sum():.1f}% (should be ~100%)")
    
    # landslide susceptibility distribution analysis
    print(f"\nLandslide Susceptibility Distribution:")
    
    total_pixels = cleaned_df_landslide['count'].sum()
    
    for idx, row in cleaned_df_landslide.iterrows():
        if row['count'] > 0:  # only show categories with data (i.e., exclude "0")
            print(f"- {row['bin']}: {row['count']:,.0f} pixels ({row['percentage']:.1f}%)")
    
    # risk level analysis
    if len(cleaned_df_landslide) > 0:
        # filter out zero-count categories
        active_categories = cleaned_df_landslide[cleaned_df_landslide['count'] > 0]
        
        if len(active_categories) > 0:
            max_susceptibility = active_categories.loc[active_categories['percentage'].idxmax()]
            min_susceptibility = active_categories.loc[active_categories['percentage'].idxmin()]
            
            print(f"\n- Most common susceptibility: {max_susceptibility['bin']} ({max_susceptibility['percentage']:.1f}%)")
            print(f"- Least common susceptibility: {min_susceptibility['bin']} ({min_susceptibility['percentage']:.1f}%)")
    
    # risk level groupings
    very_low_risk = cleaned_df_landslide[cleaned_df_landslide['bin'].isin(['Very low'])]['percentage'].sum()
    low_risk = cleaned_df_landslide[cleaned_df_landslide['bin'].isin(['Low'])]['percentage'].sum()
    medium_risk = cleaned_df_landslide[cleaned_df_landslide['bin'] == 'Medium']['percentage'].sum()
    high_risk = cleaned_df_landslide[cleaned_df_landslide['bin'].isin(['High'])]['percentage'].sum()
    very_high_risk = cleaned_df_landslide[cleaned_df_landslide['bin'].isin(['Very high'])]['percentage'].sum()
    
    print(f"\nRisk Level Summary:")
    print(f"- Very low risk areas: {very_low_risk:.1f}%")
    print(f"- Low risk areas: {low_risk:.1f}%")
    print(f"- Medium risk areas: {medium_risk:.1f}%")
    print(f"- High risk areas: {very_high_risk:.1f}%")
    print(f"- Very high risk areas: {high_risk:.1f}%")

    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values
    missing_values = cleaned_df_landslide.isnull().sum().sum()
    if missing_values > 0:
        print(f"Missing values detected: {missing_values}")
        quality_issues += 1
    
    # check for negative values (should not exist)
    negative_counts = (cleaned_df_landslide['count'] < 0).sum()
    negative_percentages = (cleaned_df_landslide['percentage'] < 0).sum()
    
    if negative_counts > 0:
        print(f"Negative count values: {negative_counts}")
        quality_issues += 1
    if negative_percentages > 0:
        print(f"Negative percentage values: {negative_percentages}")
        quality_issues += 1
    
    # check percentage sum
    percentage_sum = cleaned_df_landslide['percentage'].sum()
    if abs(percentage_sum - 100) > 0.1:
        print(f"Percentage sum deviation: {percentage_sum:.1f}% (should be ~100%)")
        quality_issues += 1
    
    # check for duplicate categories
    duplicates = cleaned_df_landslide['bin'].duplicated().sum()
    if duplicates > 0:
        print(f"Duplicate susceptibility categories: {duplicates}")
        quality_issues += 1
    
    # check for expected landslide susceptibility categories
    expected_categories = ['Very low', 'Low', 'Medium', 'High', 'Very high']
    actual_categories = cleaned_df_landslide['bin'].tolist()
    missing_categories = set(expected_categories) - set(actual_categories)
    if missing_categories:
        print(f"Categories not present in data: {missing_categories}")
    
    if quality_issues == 0:
        print("No data quality issues detected")
    
except Exception as e:
    print(f"Error processing landslide susceptibility data: {e}")
    print("Troubleshooting steps:")
    print("   1. Ensure TIF file exists at the specified path")
    print("   2. Check that the TIF file contains values 1-5 (or 0-5)")
    print("   3. Verify the TIF file is not corrupted")
    print("   4. Ensure rasterio library is installed: pip install rasterio")
    print("   5. Check file permissions and disk space")
    print("   6. If data includes value 0, try setting include_nodata=True")

print("\n" + "="*60)

# optional: to include value, "0" in analysis
# print("\n" + "="*30 + " INCLUDING VALUE 0 " + "="*30)
# try:
#     cleaned_df_with_zero = clean_ls_area(input_tif_path, 
#                                                output_file='data/processed/ls_area_with_nodata.csv',
#                                                include_nodata=True)
#     print("Alternative analysis (including value, "0") completed!")
#     print(cleaned_df_with_zero)
# except Exception as e:
#     print(f"Error in alternative analysis: {e}")

LANDSLIDE SUSCEPTIBILITY DATA PROCESSING
Analyzing TIF data structure...
Unique values in TIF: [  0   1   2 127]
Data range: 0 to 127
NoData value: 127.0
Value 0: 59 pixels
Value 1: 423 pixels
Value 2: 9 pixels
Value 127: 9 pixels

----------------------------------------
Unique values found in data: [1 2]
Cleaned landslide data saved to: data/processed/ls_area.csv
Susceptibility categories: 5
Total pixels analyzed: 432
Percentage coverage verification: 100.0% (should be ~100%)
Dominant susceptibility level: Very low (97.9%)
Landslide susceptibility data processed successfully!

Cleaned data shape: (5, 4)
Cleaned data columns: ['bin', 'susceptibility', 'count', 'percentage']

Processed Landslide Susceptibility data:
         bin susceptibility  count  percentage
0   Very low              1    423       97.92
1        Low              2      9        2.08
2     Medium              3      0        0.00
3       High              4      0        0.00
4  Very high              5      0     

### EARTHQUAKE EVENTS

### ee.csv preparation
### Observable Notebook functions/charts:
#### 1.) "plot_ee" / "chart_ee" (i.e., Significant earthquakes within 500 km since 1900)

In [ ]:
# EARTHQUAKE EVENTS - ee.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_ee" / "chart_ee" (Significant Earthquakes within 500 km since 1900)

# load "raw" (i.e. "dirty") tabular output data
raw_df_ee = pd.read_csv('data/raw/earthquake-events.csv') # updatefile path

# basic info about raw data
print("Raw earthquake events data info:")
print(f"Shape: {raw_df_ee.shape}")
print(f"Columns: {list(raw_df_ee.columns)}")
print(f"Total data points: {len(raw_df_ee)}")

# preview key data ranges if available
if 'eqMagnitude' in raw_df_ee.columns:
    print(f"Magnitude range: {raw_df_ee['eqMagnitude'].min():.1f} - {raw_df_ee['eqMagnitude'].max():.1f}")
if 'distance' in raw_df_ee.columns:
    print(f"Distance range: {raw_df_ee['distance'].min():.0f} - {raw_df_ee['distance'].max():.0f} km")

print(f"Data preview:")
print(raw_df_ee.head())
print("\n" + "="*50 + "\n")

# clean data using clean_ee function in clean.py
try:
    cleaned_df_ee = clean_ee('data/raw/earthquake-events.csv') # updatefile path
    print("Earthquake events data cleaned successfully!")
    
    # cleaned data info
    print(f"\nCleaned data shape: {cleaned_df_ee.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_ee.columns)}")
    print(f"Sample of cleaned data:")
    print(cleaned_df_ee.head(10))
    
    # basic data validation
    print(f"\nData validation:")
    print(f"- Missing values: {cleaned_df_ee.isnull().sum().sum()}")
    print(f"- Year range: {cleaned_df_ee['begin_year'].min()} - {cleaned_df_ee['begin_year'].max()}")
    print(f"- Magnitude range: {cleaned_df_ee['eqMagnitude'].min():.1f} - {cleaned_df_ee['eqMagnitude'].max():.1f}")
    print(f"- Distance range: {cleaned_df_ee['distance'].min():.0f} - {cleaned_df_ee['distance'].max():.0f} km")
    
    # earthquake data analysis
    print(f"\nEarthquake Data Summary:")
    
    # basic statistics
    avg_magnitude = cleaned_df_ee['eqMagnitude'].mean()
    avg_distance = cleaned_df_ee['distance'].mean()
    
    print(f"- Total events: {len(cleaned_df_ee)}")
    print(f"- Average magnitude: {avg_magnitude:.1f}")
    print(f"- Average distance: {avg_distance:.0f} km")
    print(f"- Standard deviation (magnitude): {cleaned_df_ee['eqMagnitude'].std():.1f}")
    print(f"- Standard deviation (distance): {cleaned_df_ee['distance'].std():.0f} km")
    
    # temporal distribution
    years_span = cleaned_df_ee['begin_year'].max() - cleaned_df_ee['begin_year'].min()
    events_per_decade = (len(cleaned_df_ee) / years_span) * 10 if years_span > 0 else 0
    
    print(f"- Time span: {years_span} years")
    print(f"- Average events per decade: {events_per_decade:.1f}")
    
    # ID extremes
    strongest_eq = cleaned_df_ee.loc[cleaned_df_ee['eqMagnitude'].idxmax()]
    closest_eq = cleaned_df_ee.loc[cleaned_df_ee['distance'].idxmin()]
    
    print(f"- Strongest earthquake: {strongest_eq['eqMagnitude']:.1f} magnitude in {strongest_eq['begin_year']}")
    print(f"- Closest earthquake: {closest_eq['distance']:.0f} km in {closest_eq['begin_year']}")
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values in key columns
    missing_magnitude = cleaned_df_ee['eqMagnitude'].isna().sum()
    missing_distance = cleaned_df_ee['distance'].isna().sum()
    missing_year = cleaned_df_ee['begin_year'].isna().sum()
    
    if missing_magnitude > 0:
        print(f"Missing magnitude values: {missing_magnitude}")
        quality_issues += 1
    if missing_distance > 0:
        print(f"Missing distance values: {missing_distance}")
        quality_issues += 1
    if missing_year > 0:
        print(f"Missing year values: {missing_year}")
        quality_issues += 1
    
    # check for impossible values
    negative_magnitude = (cleaned_df_ee['eqMagnitude'] < 0).sum()
    negative_distance = (cleaned_df_ee['distance'] < 0).sum()
    
    if negative_magnitude > 0:
        print(f"Negative magnitude values: {negative_magnitude}")
        quality_issues += 1
    if negative_distance > 0:
        print(f"Negative distance values: {negative_distance}")
        quality_issues += 1
    
    # check for duplicate events (same year, magnitude, and distance)
    duplicates = cleaned_df_ee.duplicated(subset=['begin_year', 'eqMagnitude', 'distance']).sum()
    if duplicates > 0:
        print(f"Potential duplicate events: {duplicates}")
        quality_issues += 1
    
    if quality_issues == 0:
        print("No data quality issues detected")
        
except Exception as e:
    print(f"Error cleaning earthquake events data: {e}")
    print("Check that the earthquake-events.csv file exists and has the correct format")
    print("Expected columns: BEGAN, eqMagnitude, distance, text, line1, line2, line3")

# save confirmation and next steps
if 'cleaned_df_ee' in locals():
    print(f"\nCleaned data saved to: data/processed/ee.csv")
    
    # preview of data structure
    print(f"\nData structure summary:")
    print(f"- Columns: {list(cleaned_df_ee.columns)}")
    print(f"- Time series length: {len(cleaned_df_ee)} events")
    print(f"- Year range: {cleaned_df_ee['begin_year'].min()} - {cleaned_df_ee['begin_year'].max()}")
    print(f"- Data types: {dict(cleaned_df_ee.dtypes)}")
    
else:
    print("No cleaned earthquake data available")
    print("Troubleshooting steps:")
    print("   1. Ensure earthquake-events.csv exists in data/raw/")
    print("   2. Check file has correct columns: BEGAN, eqMagnitude, distance, text, line1, line2, line3")
    print("   3. Verify data contains parseable dates in BEGAN column")
    print("   4. Check that magnitude and distance values are numeric")

### LIQUEFACTION SUSCEPTIBILITY

### l_area.csv preparation (i.e., % area with different liquefaction susceptibility levels - "No Data" (0); "Very low" (1); "Low" (2); "Medium" (3); "High" (4); and "Very high" (5))
### Observable Notebook functions/charts:
#### 1.) "plot_l_area" / "chart_l_area" (i.e., Percentage of area with different liquefaction susceptibiltiy levels, "No Data" (0); "Very low" (1); "Low" (2); "Medium" (3); "High" (4); and "Very high" (5)

In [14]:
# LIQUEFACTION SUSCEPTIBILITY - l_area.csv preparation from raw tif data for Observable Notebook plot functions/charts:
# 1.) "plot_l_area" / "chart_l_area" (i.e., Percentage of Area with different liquefaction susceptibility levels, "No Data" (0); "Very low" (1); "Low" (2); "Medium" (3); "High" (4); and "Very high" (5)

# load "raw" (i.e. "dirty") tif data
input_tif_path = 'data/raw/2025-11-mauritania-nouakchott_02-process-output_spatial_nouakchott_liquefaction.tif' # updatefile path

print("="*60)
print("LIQUEFACTION SUSCEPTIBILITY DATA PROCESSING")
print("="*60)

# data value distribution
print("Analyzing TIF data structure...")

try:
    import rasterio
    with rasterio.open(input_tif_path) as src:
        data = src.read(1)
        unique_vals = np.unique(data[~np.isnan(data)])
        print(f"Unique values in TIF: {unique_vals}")
        print(f"Data range: {data.min()} to {data.max()}")
        print(f"NoData value: {src.nodata}")
        
        # count pixels for each value
        for val in unique_vals:
            count = np.sum(data == val)
            print(f"Value {val}: {count:,} pixels")

except Exception as e:
    print(f"Could not examine TIF structure: {e}")

print("\n" + "-"*40)

# process tif file using clean_l_area function in clean.py
# set "include_nodata=False" to exclude value, "0" ("NoData"/background)
try:
    cleaned_df_liquefaction = clean_l_area(input_tif_path, include_nodata=False)
    print("Liquefaction susceptibility data processed successfully!")
    
    # cleaned data structure
    print(f"\nCleaned data shape: {cleaned_df_liquefaction.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_liquefaction.columns)}")
    print(f"\nProcessed Liquefaction Susceptibility data:")
    print(cleaned_df_liquefaction)
    
    # basic data validation
    print(f"\nData Validation:")
    print(f"- Missing values: {cleaned_df_liquefaction.isnull().sum().sum()}")
    print(f"- Susceptibility categories: {len(cleaned_df_liquefaction)}")
    print(f"- Total pixels: {cleaned_df_liquefaction['count'].sum():,.0f}")
    print(f"- Percentage sum: {cleaned_df_liquefaction['percentage'].sum():.1f}% (should be ~100%)")
    
    # liquefaction susceptibility distribution analysis
    print(f"\nLiquefaction Susceptibility Distribution:")
    
    total_pixels = cleaned_df_liquefaction['count'].sum()
    
    for idx, row in cleaned_df_liquefaction.iterrows():
        if row['count'] > 0:  # only show categories with data
            print(f"- {row['bin']}: {row['count']:,.0f} pixels ({row['percentage']:.1f}%)")
    
    # risk level analysis
    if len(cleaned_df_liquefaction) > 0:
        # filter out zero-count categories
        active_categories = cleaned_df_liquefaction[cleaned_df_liquefaction['count'] > 0]
        
        if len(active_categories) > 0:
            max_susceptibility = active_categories.loc[active_categories['percentage'].idxmax()]
            min_susceptibility = active_categories.loc[active_categories['percentage'].idxmin()]
            
            print(f"\n- Most common susceptibility: {max_susceptibility['bin']} ({max_susceptibility['percentage']:.1f}%)")
            print(f"- Least common susceptibility: {min_susceptibility['bin']} ({min_susceptibility['percentage']:.1f}%)")
    
    # risk level groupings
    very_low_risk = cleaned_df_liquefaction[cleaned_df_liquefaction['bin'].isin(['Very low'])]['percentage'].sum()
    low_risk = cleaned_df_liquefaction[cleaned_df_liquefaction['bin'].isin(['Low'])]['percentage'].sum()
    medium_risk = cleaned_df_liquefaction[cleaned_df_liquefaction['bin'] == 'Medium']['percentage'].sum()
    high_risk = cleaned_df_liquefaction[cleaned_df_liquefaction['bin'].isin(['High'])]['percentage'].sum()
    very_high_risk = cleaned_df_liquefaction[cleaned_df_liquefaction['bin'].isin(['Very high'])]['percentage'].sum()

    
    print(f"\nRisk Level Summary:")
    print(f"- Very low risk areas: {very_low_risk:.1f}%")
    print(f"- Low risk areas: {low_risk:.1f}%")
    print(f"- Medium risk areas: {medium_risk:.1f}%")
    print(f"- High risk areas: {high_risk:.1f}%")
    print(f"- Very high risk areas: {very_high_risk:.1f}%")

    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values
    missing_values = cleaned_df_liquefaction.isnull().sum().sum()
    if missing_values > 0:
        print(f"Missing values detected: {missing_values}")
        quality_issues += 1
    
    # check for negative values (should not exist)
    negative_counts = (cleaned_df_liquefaction['count'] < 0).sum()
    negative_percentages = (cleaned_df_liquefaction['percentage'] < 0).sum()
    
    if negative_counts > 0:
        print(f"Negative count values: {negative_counts}")
        quality_issues += 1
    if negative_percentages > 0:
        print(f"Negative percentage values: {negative_percentages}")
        quality_issues += 1
    
    # check percentage sum
    percentage_sum = cleaned_df_liquefaction['percentage'].sum()
    if abs(percentage_sum - 100) > 0.1:
        print(f"Percentage sum deviation: {percentage_sum:.1f}% (should be ~100%)")
        quality_issues += 1
    
    # check for duplicate categories
    duplicates = cleaned_df_liquefaction['bin'].duplicated().sum()
    if duplicates > 0:
        print(f"Duplicate susceptibility categories: {duplicates}")
        quality_issues += 1
    
    # check for expected liquefaction susceptibility categories
    expected_categories = ['Very low', 'Low', 'Medium', 'High', 'Very high']
    actual_categories = cleaned_df_liquefaction['bin'].tolist()
    missing_categories = set(expected_categories) - set(actual_categories)
    if missing_categories:
        print(f"Categories not present in data: {missing_categories}")
    
    if quality_issues == 0:
        print("No data quality issues detected")
    
except Exception as e:
    print(f"Error processing liquefaction susceptibility data: {e}")
    print("Troubleshooting steps:")
    print("   1. Ensure TIF file exists at the specified path")
    print("   2. Check that the TIF file contains values 1-5 (or 0-5)")
    print("   3. Verify the TIF file is not corrupted")
    print("   4. Ensure rasterio library is installed: pip install rasterio")
    print("   5. Check file permissions and disk space")
    print("   6. If data includes value 0, try setting include_nodata=True")

print("\n" + "="*60)

# optional: to include value, "0" in analysis
# print("\n" + "="*30 + " INCLUDING VALUE 0 " + "="*30)
# try:
#     cleaned_df_with_zero = clean_liquefaction_area(input_tif_path, 
#                                                   output_file='data/processed/l_area_with_nodata.csv',
#                                                   include_nodata=True)
#     print("Alternative analysis (including value 0) completed!")
#     print(cleaned_df_with_zero)
# except Exception as e:
#     print(f"Error in alternative analysis: {e}")

LIQUEFACTION SUSCEPTIBILITY DATA PROCESSING
Analyzing TIF data structure...
Unique values in TIF: [  0   3   4 255]
Data range: 0 to 255
NoData value: 255.0
Value 0: 42 pixels
Value 3: 121 pixels
Value 4: 158 pixels
Value 255: 15 pixels

----------------------------------------
Unique values found in data: [3 4]
Cleaned liquefaction data saved to: data/processed/l_area.csv
Susceptibility categories: 5
Total pixels analyzed: 279
Percentage coverage verification: 100.0% (should be ~100%)
Dominant susceptibility level: High (56.6%)
Liquefaction susceptibility data processed successfully!

Cleaned data shape: (5, 4)
Cleaned data columns: ['bin', 'susceptibility', 'count', 'percentage']

Processed Liquefaction Susceptibility data:
         bin susceptibility  count  percentage
0   Very low              1      0        0.00
1        Low              2      0        0.00
2     Medium              3    121       43.37
3       High              4    158       56.63
4  Very high              5  

### FIRE WEATHER INDEX (FWI)

### fwi.csv
### Observable Notebook functions/charts:
#### 1.) "plot_fwi" / "chart_fwi" (i.e., Fire Weather Index (FWI), January - December)
#### 2.) "plot_fwi_d" / "chart_fwi_d" (Fire Weather Index (FWI) Danger Level Distribution)


In [15]:
# FIRE WEATHER INDEX - fwi.csv preparation for Observable Notebook plot functions/charts:
# 1.) "plot_fwi"/ "chart_fwi" (Fire Weather Index (FWI), January - December)
# 2.) "plot_fwi_d" / "chart_fwi_d" (Fire Weather Index (FWI) Danger Level Distribution)

# load "raw" (i.e. "dirty") tabular output data
raw_df_fwi = pd.read_csv('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_fwi.csv') # updatefile path

# basic info about raw data
print("Raw fire weather index data info:")
print(f"Shape: {raw_df_fwi.shape}")
print(f"Columns: {list(raw_df_fwi.columns)}")
print(f"Total data points: {len(raw_df_fwi)}")

# preview key data ranges if available
if 'week' in raw_df_fwi.columns:
    print(f"Week range: {raw_df_fwi['week'].min()} - {raw_df_fwi['week'].max()}")
if 'pctile_95' in raw_df_fwi.columns:
    print(f"FWI range: {raw_df_fwi['pctile_95'].min():.2f} - {raw_df_fwi['pctile_95'].max():.2f}")

print(f"Data preview:")
print(raw_df_fwi.head())
print("\n" + "="*50 + "\n")

# clean data using clean_fwi function in clean.py
try:
    cleaned_df_fwi = clean_fwi('data/raw/2025-11-mauritania-nouakchott_02-process-output_tabular_nouakchott_fwi.csv') # updatefile path
    print("Fire weather index data cleaned successfully!")
    
    # cleaned data info
    print(f"\nCleaned data shape: {cleaned_df_fwi.shape}")
    print(f"Cleaned data columns: {list(cleaned_df_fwi.columns)}")
    print(f"Sample of cleaned data:")
    print(cleaned_df_fwi.head(10))
    
    # basic data validation
    print(f"\nData validation:")
    print(f"- Missing values: {cleaned_df_fwi.isnull().sum().sum()}")
    print(f"- Week coverage: {len(cleaned_df_fwi)} weeks")
    print(f"- Week range: {cleaned_df_fwi['week'].min()} - {cleaned_df_fwi['week'].max()}")
    print(f"- FWI range: {cleaned_df_fwi['fwi'].min():.2f} - {cleaned_df_fwi['fwi'].max():.2f}")
    
    # fire weather analysis
    print(f"\nFire Weather Data Summary:")
    
    # basic statistics
    avg_fwi = cleaned_df_fwi['fwi'].mean()
    median_fwi = cleaned_df_fwi['fwi'].median()
    std_fwi = cleaned_df_fwi['fwi'].std()
    
    print(f"- Total weeks: {len(cleaned_df_fwi)}")
    print(f"- Average FWI: {avg_fwi:.2f}")
    print(f"- Median FWI: {median_fwi:.2f}")
    print(f"- Standard deviation: {std_fwi:.2f}")
    
    # ID extremes
    peak_week = cleaned_df_fwi.loc[cleaned_df_fwi['fwi'].idxmax()]
    lowest_week = cleaned_df_fwi.loc[cleaned_df_fwi['fwi'].idxmin()]
    
    print(f"- Highest FWI: {peak_week['fwi']:.2f} (Week {peak_week['week']}, {peak_week['monthName']}, {peak_week['danger']})")
    print(f"- Lowest FWI: {lowest_week['fwi']:.2f} (Week {lowest_week['week']}, {lowest_week['monthName']}, {lowest_week['danger']})")
    
    # danger level distribution
    danger_counts = cleaned_df_fwi['danger'].value_counts()
    print(f"\nDanger Level Distribution:")
    for level in ['Very low', 'Low', 'Moderate', 'High', 'Very high', 'Extreme']:
        count = danger_counts.get(level, 0)
        percentage = (count / len(cleaned_df_fwi)) * 100
        print(f"- {level}: {count} weeks ({percentage:.1f}%)")
    
    # monthly statistics
    monthly_stats = cleaned_df_fwi.groupby('monthName')['fwi'].agg(['mean', 'max', 'min']).round(2)
    
    print(f"\nMonthly FWI Statistics:")
    for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:
        if month in monthly_stats.index:
            stats = monthly_stats.loc[month]
            print(f"- {month}: Mean {stats['mean']:.2f}, Range {stats['min']:.2f} - {stats['max']:.2f}")
    
    # data quality checks
    print(f"\nData Quality Checks:")
    
    quality_issues = 0
    
    # check for missing values in key columns
    missing_week = cleaned_df_fwi['week'].isna().sum()
    missing_fwi = cleaned_df_fwi['fwi'].isna().sum()
    missing_month = cleaned_df_fwi['monthName'].isna().sum()
    missing_danger = cleaned_df_fwi['danger'].isna().sum()
    
    if missing_week > 0:
        print(f"Missing week values: {missing_week}")
        quality_issues += 1
    if missing_fwi > 0:
        print(f"Missing FWI values: {missing_fwi}")
        quality_issues += 1
    if missing_month > 0:
        print(f"Missing month values: {missing_month}")
        quality_issues += 1
    if missing_danger > 0:
        print(f"Missing danger values: {missing_danger}")
        quality_issues += 1
    
    # check for impossible values
    negative_fwi = (cleaned_df_fwi['fwi'] < 0).sum()
    if negative_fwi > 0:
        print(f"Negative FWI values: {negative_fwi}")
        quality_issues += 1
    
    # check for week sequence
    expected_weeks = set(range(1, 54))  # (Note: 53 weeks in a year) 
    actual_weeks = set(cleaned_df_fwi['week'].unique())
    missing_weeks = expected_weeks - actual_weeks
    if len(missing_weeks) > 0:
        print(f"Missing weeks: {sorted(list(missing_weeks))}")
        quality_issues += 1
    
    # check for duplicate weeks
    duplicates = cleaned_df_fwi['week'].duplicated().sum()
    if duplicates > 0:
        print(f"Duplicate week entries: {duplicates}")
        quality_issues += 1
    
    # check for valid danger categories
    valid_dangers = {'Very low', 'Low', 'Moderate', 'High', 'Very high', 'Extreme', 'Unknown'}
    invalid_dangers = set(cleaned_df_fwi['danger'].unique()) - valid_dangers
    if len(invalid_dangers) > 0:
        print(f"Invalid danger categories: {invalid_dangers}")
        quality_issues += 1
    
    if quality_issues == 0:
        print("No data quality issues detected")
        
except Exception as e:
    print(f"Error cleaning fire weather index data: {e}")
    print("Check that the FWI CSV file exists and has the correct format")
    print("Expected columns: week, pctile_95")

# save confirmation and next steps
if 'cleaned_df_fwi' in locals():
    print(f"\nCleaned data saved to: data/processed/fwi.csv")
    
    # preview of data structure
    print(f"\n Data structure summary:")
    print(f"- Columns: {list(cleaned_df_fwi.columns)}")
    print(f"- Time series length: {len(cleaned_df_fwi)} weeks")
    print(f"- Data types: {dict(cleaned_df_fwi.dtypes)}")
    print(f"- FWI value range: {cleaned_df_fwi['fwi'].min():.2f} - {cleaned_df_fwi['fwi'].max():.2f}")
    
else:
    print("No cleaned fire weather data available")
    print("Troubleshooting steps:")
    print("   1. Ensure FWI CSV file exists in data/raw/")
    print("   2. Check file has correct columns: week, pctile_95")
    print("   3. Verify week numbers are sequential (1-53)")
    print("   4. Check that FWI values are numeric and non-negative")

Raw fire weather index data info:
Shape: (53, 2)
Columns: ['week', 'pctile_95']
Total data points: 53
Week range: 1 - 53
FWI range: 55.25 - 137.12
Data preview:
   week   pctile_95
0     1  112.633472
1     2  113.747966
2     3  115.733023
3     4  112.483855
4     5  125.499134


Cleaned data saved to: data/processed/fwi.csv
Weeks covered: 53 weeks
Week range: 1 - 53
FWI range: 55.25 - 137.12
Danger level distribution:
  Very low: 0 weeks (0.0%)
  Low: 0 weeks (0.0%)
  Moderate: 0 weeks (0.0%)
  High: 0 weeks (0.0%)
  Very high: 0 weeks (0.0%)
  Extreme: 53 weeks (100.0%)
Peak fire weather month: Feb (max FWI: 137.12)
Fire weather index data cleaned successfully!

Cleaned data shape: (53, 4)
Cleaned data columns: ['week', 'monthName', 'fwi', 'danger']
Sample of cleaned data:
   week monthName     fwi   danger
0     1       Jan  112.63  Extreme
1     2       Jan  113.75  Extreme
2     3       Jan  115.73  Extreme
3     4       Jan  112.48  Extreme
4     5       Feb  125.50  Extreme
5 